In [24]:
import pandas as pd
from urlparse import urlparse
import tldextract
from collections import Counter
import numpy as np
import json
import math
import re

In [25]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation

In [26]:
import math
import re
from sklearn.feature_extraction import stop_words

In [27]:
dsi_stop_words = ['the', 'blog', 'i', 'in', 'new', 'use', 'a', 'how', 'it', 'like', 'need', 'sign', 'for', 
                  'rss', 'videos', 'view', 'using', 'interview', 'follow', 'read', 'make', 'video',
                  'post', 'comment', 'comments', 'subscribe', 'things', 'just', 'add', 'wise', 'know', 'upcoming', 
                  'people', 'practitioner', 'used', 'developers', 'events', 'companies', 'better', 'terms', 'time',
                  'customer', 'conference']

In [28]:
class DistanceGenerator (object):
    def __init__(self, df):
        self.df = df
    
    def calculate_distance (self, v1, v2):
        #print type(v1), type(v2)        
        if type(v1) == dict and type(v2) == dict:
            vec1 = v1
            vec2 = v2
        else:
            vec1 = eval(v1)
            vec2 = eval(v2)

        #print type(vec1), type(vec2)
        #print vec2.keys()
        #print vec1.keys()
        intersection = set(vec1.keys()) & set(vec2.keys())
        #print intersection
        numerator = sum([vec1[x] * vec2[x] for x in intersection])

        sum1 = sum([vec1[x]**2 for x in vec1.keys()])
        sum2 = sum([vec2[x]**2 for x in vec2.keys()])
        denominator = math.sqrt(sum1) * math.sqrt(sum2)

        if not denominator:
            return 0.0
        else:
            return float(numerator) / denominator
    
    def get_max_distances_idx(self, vec):
        self.df['check_distances'] =  self.df.signature.map(lambda x: self.calculate_distance(vec, x))
        dist = self.df['check_distances'].values.argsort()[::-1]
        return dist
    
    def dsi_title_letters_removal(self, x):
        s = re.sub('[^a-zA-Z]+', '', x)
        if(len(s) > 0):
            s = s.strip().lower()
            if s not in stop_words.ENGLISH_STOP_WORDS and s not in dsi_stop_words:
                return s
            else:
                return None
        return None

    def calculate_signature(self, docs):
        if len(docs) == 1:
            words = docs.split()
            flatten = [dsi_title_letters_removal(item) for item in words \
                       if dsi_title_letters_removal(item) != None]

        else:
            docs_split = [item.split() for item in docs]
            flatten = [self.dsi_title_letters_removal(item) for sublist in docs_split for item in sublist \
                       if self.dsi_title_letters_removal(item) != None]

        count = Counter(flatten)
        top_50 = count.most_common(50)
        words, cnt = zip(*top_50)
        tot = sum(cnt)
        fq = [float(i)*1.0/tot for i in cnt]
        sign = dict(zip(words, fq))
        return sign   


In [30]:
df = pd.read_csv('../data/raw/solr-dsi-domain_only-04-05-2017-cleaned.csv')

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 292147 entries, 0 to 292146
Data columns (total 12 columns):
_version_    292147 non-null int64
cache        3956 non-null object
segment      292147 non-null int64
digest       292147 non-null object
tstamp       292147 non-null object
url          292147 non-null object
anchor       31946 non-null object
content      292147 non-null object
id           292147 non-null object
title        292147 non-null object
boost        292147 non-null float64
domain       292147 non-null object
dtypes: float64(1), int64(2), object(9)
memory usage: 26.7+ MB


In [32]:
df = df.dropna(subset =['key_words'])

KeyError: ['key_words']

In [263]:
d = df['domain']
count = Counter(d)
for t in count.iteritems():
    if t[0] == 'datareview.info':
        print t[1]

In [194]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86234 entries, 0 to 86233
Data columns (total 18 columns):
url                86234 non-null object
domain             86234 non-null object
body               86234 non-null object
title              86234 non-null object
key_words          86234 non-null object
sim                86234 non-null float64
dl_sim             86234 non-null float64
analytics_sim      86234 non-null float64
dm_sim             86234 non-null float64
ml_sim             86234 non-null float64
dsj_sim            86234 non-null float64
dsn_sim            86234 non-null float64
ba_sim             86234 non-null float64
ds_sim             86234 non-null float64
summary            86234 non-null object
signature          86234 non-null object
check_distances    86234 non-null float64
bd_sim             86234 non-null float64
dtypes: float64(11), object(7)
memory usage: 11.8+ MB


In [195]:
df['key_words'] = df.key_words.map(lambda x: eval(x) )

In [196]:
tmp = df[df['domain'] != 'datareview.info']

In [197]:
tmp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 86234 entries, 0 to 86233
Data columns (total 18 columns):
url                86234 non-null object
domain             86234 non-null object
body               86234 non-null object
title              86234 non-null object
key_words          86234 non-null object
sim                86234 non-null float64
dl_sim             86234 non-null float64
analytics_sim      86234 non-null float64
dm_sim             86234 non-null float64
ml_sim             86234 non-null float64
dsj_sim            86234 non-null float64
dsn_sim            86234 non-null float64
ba_sim             86234 non-null float64
ds_sim             86234 non-null float64
summary            86234 non-null object
signature          86234 non-null object
check_distances    86234 non-null float64
bd_sim             86234 non-null float64
dtypes: float64(11), object(7)
memory usage: 12.5+ MB


In [87]:
tmp.to_csv('../data/sim/dsi_sim_v1.csv', encoding='utf-8', index=False)

In [22]:
df.head()

,url,domain,body,title,key_words,sim
0,http://www.ecmlpkdd2015.org/global-info/organi...,ecmlpkdd2015.org,Organization Committee | ECMLPKDD2015 Skip to ...,Organization Committee | ECMLPKDD2015,"{u'industrial': 0.0108303249097, u'global': 0....",0.001585
1,http://www.economist.com/,economist.com,"The Economist - World News, Politics, Economic...","The Economist - World News, Politics, Economic...","{u'radio': 0.0121951219512, u'economist': 0.13...",0.055683
2,http://www.economist.com/films,economist.com,Films | The Economist Our cookie policy has ch...,Films | The Economist,"{u'tomorrows': 0.00847457627119, u'global': 0....",0.009322
3,http://www.economist.com/news/christmas-specia...,economist.com,The king of con-men | The Economist Our cookie...,The king of con-men | The Economist,"{u'raised': 0.0268817204301, u'london': 0.0161...",0.020602
4,http://www.economist.com/news/christmas/215685...,economist.com,The song of Song | The Economist Our cookie po...,The song of Song | The Economist,"{u'chinese': 0.0414507772021, u'years': 0.0310...",0.018020


In [198]:
with open('../data/footprint/dsi_fp.json') as data_file:    
    dsi_dict = json.load(data_file)

In [199]:
def main_topic_per_doc(W):
    '''
    Output a list that holds the top topic in each document
    '''
    main_topic = []
    for doc_num, topic_row in enumerate(W):
        main_topic.append(topic_row.argsort()[::-1][0])
        #print("Article %d:" % doc_num, "Main Topic: %d" % topic_row.argsort()[::-1][0])
    return main_topic

In [200]:
def print_top_words(model, feature_names, n_top_words, df):
    for topic_idx, topic in enumerate(model.components_):
        tmp = df[df['topic'] == topic_idx]
        print("Topic #%d:" % topic_idx, "total = %d" % len(tmp))
        print(" ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))

In [201]:
key_words_vec = df['key_words']

In [202]:
key_words = [' '.join(item) for item in key_words_vec]

In [203]:
n_samples = 2000
n_features = 1000
n_topics = 10
n_top_words = 20

In [204]:
#with normalized vector
normalized_vectorizer = TfidfVectorizer(stop_words = 'english', norm=None)
normalized_document_term_mat = normalized_vectorizer.fit_transform(key_words)
words = normalized_vectorizer.get_feature_names()

In [205]:
nmf = NMF(n_components = n_topics)
nmf_W = nmf.fit_transform(normalized_document_term_mat)
nmf_H = nmf.components_

In [206]:
df['topic'] = main_topic_per_doc(nmf_W)
print_top_words(nmf, words, n_top_words, df)

('Topic #0:', 'total = 23876')
solutions services business support products management data software company customers partners contact technology security service product platform enterprise cloud resources
('Topic #1:', 'total = 1781')
unblock unfollow blocked mute pending wont retweeted retweets replies trke dansk retweet tweets following photos bahasa verified reply saved joined
('Topic #2:', 'total = 8559')
kdnuggets machine learning mining deep stories science jan scientist news vs dec nov python analytics big questions scientists data skills
('Topic #3:', 'total = 3175')
thinkers workforce followers best location worlds guidelines social sentiment knowledge badge media text practices privacy mobility intelligence policy hardware profile
('Topic #4:', 'total = 4025')
chats ibm bytes meetups explore streaming presentations infographics content facebook heroes blogs white type hub podcasts webcasts galleries cases linkedin
('Topic #5:', 'total = 6256')
central tim matteson added no

In [207]:
def get_cosine(vec1, vec2):
    intersection = set(vec1.keys()) & set(vec2.keys())
    #print intersection
    numerator = sum([vec1[x] * vec2[x] for x in intersection])

    sum1 = sum([vec1[x]**2 for x in vec1.keys()])
    sum2 = sum([vec2[x]**2 for x in vec2.keys()])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)

    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator

In [208]:
dl_dict = dict()
dl_dict['deep'] = 0.5
dl_dict['learning'] = 0.5

In [209]:
def map_func(x):
    s = x.split(',')
    t = [r.split(':')[0] for r in s]
    print ' '.join(t)

In [210]:
df['dl_sim'] =  df.key_words.map(lambda x: get_cosine(dl_dict, x))

In [211]:
dl_sim = df['dl_sim'].values.argsort()[::-1]

In [212]:
idx = 0
for i in dl_sim:
    if df.iloc[i]['dl_sim'] > 0.30:
        title = df.iloc[i]['title']
        sim_val = df.iloc[i]['dl_sim']
        idx += 1
        print idx, i, title, sim_val

1 75416 Deep Learning 0.884616271171
2 68756 RE•WORK | Blog - Deep Reinforcement Learning for Robotics 0.880075837466
3 82058 Tutorials « Deep Learning 0.864529934867
4 16372 Evolution of Deep learning models - Data Science Central 0.859136966457
5 68753 RE•WORK | Blog - Deep Learning with Structure 0.857028480621
6 68776 RE•WORK | Blog 0.854379581339
7 68767 RE•WORK | Blog 0.854195864917
8 83241 Deep Learning – important resources for learning and understanding 0.842537093044
9 83998 Deep Learning | NVIDIA Developer 0.827837354385
10 44292 Where to Learn Deep Learning – Courses, Tutorials, Software 0.820276163245
11 68755 RE•WORK | Blog - Deep Learning with Google Brain's Oriol Vinyals 0.800890373119
12 71490 Does Deep Learning Come from the Devil? 0.799329839572
13 83476 Deep Learning with Structure – a preview 0.797241005179
14 68764 RE•WORK | Blog 0.794316427078
15 39831 What To Expect from Deep Learning in 2016 and Beyond 0.791886090054
16 83263 Will Deep Learning take over Machin

178 71523 TensorFlow Disappoints – Google Deep Learning falls shallow 0.552941233077
179 78318 Juergen Schmidhuber's answers from the Reddit AMA - FastML 0.552811182076
180 72062 Andrew Ng 0.55237670873
181 79472 Andrej Karpathy Academic Website 0.551324771088
182 75539 Jurgen Schmidhuber 0.550152841465
183 68836 RE•WORK | Sponsor Us 0.550091331338
184 71691 Top stories for Jan 3-9: 20 Questions to Detect Fake Data Scientists; 5 More arXiv Deep Learning Pap 0.549744032678
185 83505 Talking Machine – Deep Learning in Speech Recognition 0.54632448447
186 71629 KDnuggets™ News 15:n20, Jun 24: Data Science Battle: R vs Python; Popular Deep Learning Tools 0.544984330244
187 68815 RE•WORK 0.544581148637
188 4714 ConvNetJS: Deep Learning in your browser 0.543702132825
189 20798 John Langford 0.541666666667
190 43788 eLmL 2015 0.540295903842
191 43897 eLmL 2016 0.540295903842
192 70525 Machine Learning 0.538642178555
193 83455 Top /r/MachineLearning Posts, Apr 5-11: Amazon Machine Learning, Nu

390 82307 HPC Archives - insideBIGDATA 0.435859627376
391 83151 Top KDnuggets tweets, Nov 18-19: Must read: Deep Learning 101; “Pythonization” of data analysis 0.435610281892
392 71630 KDnuggets™ News 15:n23, Jul 22: Deep Learning Adversarial Examples Myths & Facts; Stop Hiring Data S 0.434967907198
393 44100 H2O World, Open Source Machine Learning Meeting, Nov 18-19, Mountain View 0.43482610433
394 68772 RE•WORK | Blog 0.434539598671
395 45690 Top stories for Nov 8-14: 5 Best Machine Learning APIs; A Statistical View of Deep Learning 0.43449688746
396 83273 Course: Practical Machine Learning for Engineers, New York, Nov 10-12 0.434194184857
397 68829 RE•WORK 0.434121571062
398 44268 Interview: Arno Candel, H20.ai on How to Quick Start Deep Learning with H2O 0.434018539953
399 71645 Attention and Memory in Deep Learning and NLP 0.432992934587
400 41080 Top stories for Jul 12-18: 50+ Data Science, Machine Learning Cheat Sheets; Deep Learning Adversaria 0.432580337629
401 82057 11-785 DE

676 80574 Simple Machine Learning for the Masses at PAPIs '15 | Lanyrd 0.361813613493
677 83389 More Free Data Mining, Data Science Books and Resources 0.361751107431
678 45905 arXiv.org and the 24 Hour Research Cycle 0.361478445646
679 68827 RE•WORK 0.361157559257
680 40027 Quantitative Research Analyst 0.360875158789
681 45693 2015 Nov Top Tweets 0.360510438939
682 40029 Statistical Modeling Analyst 0.360485234138
683 45741 Top 10 Machine Learning Projects on Github 0.360484727247
684 72112 C++ 0.360225985464
685 83052 LIONbook Chapter 7: Ranking and selecting features 0.360156558816
686 43524 NASA Asteroid Grand Challenge 0.360024482497
687 43301 Top /r/MachineLearning Posts, Feb 8-14: Automating Tinder, Statistics and Machine Learning 0.359775210718
688 71542 Top stories for Nov 22-28: TensorFlow Disappoints; How Applications of Big Data Drive Industries 0.359551896866
689 33418 The Believers - The Chronicle of Higher Education 0.359452252654
690 71678 2016 Jan Publications 0.35945

995 83005 Cartoon: Mother Of All Data 0.32034013337
996 83035 The Age of Big Data – BBC Documentary 0.320340133369
997 44114 SPOTLIGHT: Can Data Science Save Humanity from Mosquitoes and other Deadly Insects? #2 0.320329564309
998 6092 Home Page of Geoffrey Hinton 0.320256307611
999 45999 Backpropagation 0.32025630761
1000 71673 2016 Jan Opinions, Interviews, Reports 0.320221844309
1001 44458 Taming the Internet of Things – KNIME Case Study 0.320199297141
1002 68781 RE•WORK | Blog 0.320104347314
1003 39887 U. Virginia: MS / PhD positions in Machine Learning, Data Science 0.320023803109
1004 82999 KDnuggets Cartoon looks at IRS, Big Data, and Taxes 0.320005224618
1005 31660 BigML.io | The Official Blog of BigML.com 0.319946537845
1006 47175 WEBINAR:Customer Analytics Roadmap 0.319881639108
1007 83277 2014 Nov News, Features 0.31979810796
1008 44275 Info Kit: Statistics, Predictive Modeling and Data Mining with JMP 0.319791008241
1009 47120 BPDM scholarships Broadening Participation in D

In [213]:
tmp = df[df['dl_sim'] > 0.3]
docs = tmp['body']
aG = DistanceGenerator(tmp)
sign = aG.calculate_signature(docs)

In [214]:
df['dl_sim'] =  df.key_words.map(lambda x: get_cosine(sign, x))

In [215]:
analytics_dict = dict()
analytics_dict['analytics'] = 1.0
df['analytics_sim'] =  df.key_words.map(lambda x: get_cosine(analytics_dict, x))
analytics_sim = df['analytics_sim'].values.argsort()[::-1]

In [216]:
idx = 0
for i in analytics_sim:
    if df.iloc[i]['analytics_sim'] > 0.40:
        title = df.iloc[i]['title']
        sim_val = df.iloc[i]['analytics_sim']
        idx += 1
        print idx, i, title, sim_val

1 722 Comments on: Analytics 1.0
2 18948 Comments on: Analytics1™ 1.0
3 43170 DATA ANALYTICS 2012 0.91118874576
4 11320 Analytics at Work « Analytics Training Institute 0.87678827979
5 81623 Big Data Analytics | Marketing Analytics | Customer Analytics | Absolutdata 0.832050294338
6 11171 Archives 0.823762069242
7 82562 The Authority on Analytics Maturity and Best Practices - International Institute for Analytics 0.823713950893
8 80271 The Authority on Analytics Maturity and Best Practices - International Institute for Analytics 0.823713950893
9 54578 customer retention | The Big Data Hub 0.805718332075
10 64767 Using an Analytical Approach to Find Analytics Talent | Talent Analytics, Corp. 0.803434736585
11 6643 Solutions | Fuzzy Logix 0.798823018395
12 31950 Online Analytics Training | Jigsaw Academy Blog on Analytics 0.798453475687
13 22464 Analytics 3.0 - International Institute for Analytics 0.798198729717
14 35464 Analytics 3.0 - International Institute for Analytics 0.7981987297

178 57331 Analytics Job - Digital Marketing and Analytics by Anil Batra 0.666666666665
179 72037 Advanced Analytics 0.666611563176
180 75866 Digital Analytics | Marketing Analytics | Big Data | Web Analytics 0.666027440896
181 82389 budgeting | IBM Big Data & Analytics Hub 0.665750285935
182 6673 Predictive Campaign Marketing | Fuzzy Logix 0.665512264645
183 57332 Resources - Digital Marketing and Analytics by Anil Batra 0.665261408165
184 11275 Productization of Analytics | Cypher 2015 0.665190105238
185 82983 Analytics Education Boom – Trends and Overview 0.665117142391
186 10278 Predictive Analytics World San Francisco 2015 - AnalyticBridge 0.664845439424
187 10266 Predictive Analytics World Boston 2014 - AnalyticBridge 0.664692417256
188 35798 cloud analytics | The Big Data Hub 0.664638918755
189 11150 Advertising 0.663952809568
190 81566 Predictive analytics and business intelligence software and solutions | Angoss www.angoss.com 0.663913729924
191 5471 Google Analytics - Mobile, 

393 74270 Predictive Analytics Times interviews Field Cady 0.638401339361
394 74230 Automation: Friend or Foe to the Predictive Analytics Practitioner 0.638401339361
395 74287 Predictive Analytics World Healthcare interviews Scott Jelinsky 0.638150715461
396 54044 Financial Performance Management | The Big Data Hub 0.638053287228
397 74282 Predictive Analytics World interviews Madhusudan Raman 0.637955147117
398 74278 Predictive Analytics World interviews Herman Jopia 0.637955147117
399 74257 Predictive Analytics World in Color 0.637955147117
400 74293 Wise Practitioner - Predictive Workforce Analytics Interview Series: Lisa Disselkamp and Tristan Aub 0.637921381034
401 74284 Predictive Analytics World Boston interview Michael Berry 0.637907734896
402 74239 From Code to Reports with knitr & Markdown 0.637848262712
403 74286 PAW GOV interviews Patty Larson, Co-Director, Nat'l Insider Threat Task Force 0.637831897324
404 74229 A Rogue Liberal: Halting NSA Bulk Data Collection Compromises

694 26133 statistics | Analyst Perspective - Tony Cosentino 0.604518662997
695 74313 Predictive Analytics World Workforce: March - April, 2014 0.604489899814
696 78054 wearables | IBM Big Data & Analytics Hub 0.604386846286
697 54569 law enforcement | The Big Data Hub 0.604257127886
698 23230 Cloud for Analytics | Business Analytics 0.604161236043
699 35733 financial services | The Big Data Hub 0.604077101925
700 78048 i2 Summit | IBM Big Data & Analytics Hub 0.604009591155
701 54053 Transform Financial Processes | The Big Data Hub 0.603999640286
702 21752 Business Uses of Predictive Analytics - insideBIGDATA 0.603703429943
703 10414 Bobbe Cook's Page - AnalyticBridge 0.603644331186
704 71209 Useful Links for Analytics Students | Analytics and Data Mining Programs 0.603582858982
705 64875 enhanced ecommerce analytics Archives | Tatvic Blog 0.603307424841
706 62084 Book: Web Analytics: An Hour a Day - by Avinash Kaushik 0.603022689156
707 35596 IBM Watson Analytics | The Big Data Hub 0.

1017 11284 Anshuman Gupta | Cypher 2015 0.55648667491
1018 64763 Form an Alliance with Talent Analytics, Corp. | Talent Analytics, Corp. 0.55645004346
1019 53732 Agile Analytics: Empowering small and midsize businesses 0.556414884076
1020 26091 cognos analytics | Analyst Perspective - Tony Cosentino 0.556379295616
1021 74474 Google Analytics for Advertisers 0.556337142014
1022 21943 Smart Guest Analytics 2015 0.556293911167
1023 4880 Operational Analytics articles, case studies, and infographics 0.556242699362
1024 82392 public safety | IBM Big Data & Analytics Hub 0.556199678156
1025 43072 IAPA - Institute of Analytics Professionals of Australia 0.555984721749
1026 42731 Disclaimers & Disclosures - Occam's Razor by Avinash Kaushik 0.555959449144
1027 78167 listicle | IBM Big Data & Analytics Hub 0.555959449143
1028 16700 Resources - Decision Systems 0.555839862252
1029 43996 IBM Analytics - Advanced Analytics - United States 0.55583986225
1030 23405 Big Data | 0.555701068581
1031 2337

...

2398 35073 Basin Electric Power Cooperative | The Big Data Hub 0.452637622446
2399 36462 David Birmingham | The Big Data Hub 0.45251858113
2400 54419 Create your own luck with analytics | The Big Data Hub 0.452508118803
2401 34969 How athenahealth gains insight into business performance | The Big Data Hub 0.452472733055
2402 82406 IoT manufacturers | IBM Big Data & Analytics Hub 0.4524093511
2403 36192 governance | The Big Data Hub 0.452308137658
2404 42742 actionable analytics - Occam's Razor by Avinash Kaushik 0.452267016867
2405 77720 Quick and efficient claim resolution with predictive analytics | IBM Big Data & Analytics Hub 0.45225278098
2406 72023 SaaS Analytics Solutions 0.452186735155
2407 36015 cyber threats | The Big Data Hub 0.452124323374
2408 36716 Demystify your data with the next generation of business analytics | The Big Data Hub 0.452026299906
2409 56563 Analytics | The ROI Revolution Blog 0.452013432873
2410 54157 6 ways to give insurance customers what they want | T

2748 36383 Predictive Analytics World Conference: Agenda At a Glance - San Francisco 2016 0.435868612226
2749 62210 Social Analytics | Twitter, Facebook, YouTube Measurement | Webtrends 0.435859627375
2750 61490 Tom H. C. Anderson - Next Gen Market Research™ 0.435787768562
2751 83214 New Book: Analytics in a Big Data World – The Essential Guide to Data Science 0.435579519533
2752 10576 Michael Mortenson's Page - AnalyticBridge 0.435546177732
2753 35020 ManpowerGroup gains deep industry insights through advanced analytics | The Big Data Hub 0.435464843161
2754 46528 Datalicious: Head of Analytics 0.435413082451
2755 60467 Data Scientist Career Path | Burtch Works 0.435285750066
2756 57193 TheMainstreamSeer: Predictive Analytics Demystified 0.435285750066
2757 26957 Industries - comScore, Inc 0.435194139889
2758 26962 Telecommunications - comScore, Inc 0.435194139889
2759 533 Xmedia - comScore, Inc 0.435194139889
2760 26961 Technology - comScore, Inc 0.435194139889
2761 26958 Education -

3107 65199 March 2015 in San Francisco | Analytics Workshops | Text Analytics World 0.426098913033
3108 75742 Tom Davenport 0.426033051275
3109 45175 Pasha Roberts 0.425971375685
3110 67042 Financial Performance Management | IBM Big Data & Analytics Hub 0.425971375685
3111 36689 Customer analytics: Using data to improve your bottom line | The Big Data Hub 0.425812073158
3112 54205 What's your strategy for improving critical asset performance? | The Big Data Hub 0.425799716854
3113 12910 C-Suite Archives - Burtch Works 0.425799123238
3114 56611 Training | The ROI Revolution Blog 0.425657589151
3115 54159 Analytics helps win the automotive security race | The Big Data Hub 0.425628265379
3116 75378 Customer Engagement 0.425596141279
3117 77647 IBM Insight 2015, day 1: Observations, announcements and insights | IBM Big Data & Analytics Hub 0.425576923781
3118 11451 Success Stories | Angoss www.angoss.com 0.425526762184
3119 36352 Advanced Methods Hands-on: Predictive Modeling Techniques - 

3458 35781 IBM System z | The Big Data Hub 0.419230992559
3459 8438 Corporate Training - Edvancer Eduventures 0.41922180815
3460 72957 | SmartData Collective 0.41917178955
3461 82367 enterprise data hubs | IBM Big Data & Analytics Hub 0.419171789549
3462 44424 Analytics Team Dissolution – Unique Case Study 0.419122072892
3463 16681 Application Development - Decision Systems 0.419079038063
3464 11285 Atul Jalan | Cypher 2015 0.419079038063
3465 81487 Seven Questions on Adopting Analytics Culture - AnalyticBridge 0.419072004588
3466 28117 Company - CBIG Consulting 0.419026240703
3467 77675 Jen Q. Public: Can analytics help fill the cracks in our education systems? | IBM Big Data & Analyti 0.418944657704
3468 23429 Information Management | Analyst Perspective - Tony Cosentino 0.418814614337
3469 16685 Budgeting & Planning - Decision Systems 0.418802887656
3470 46702 Creating a Global Analytics Culture - San Diego, February 12-13 - Data Science Central 0.418762843194
3471 18089 What is you

3776 43342 Interview: Slava Akmaev, Berg on Challenges in Transitioning Analytics to Clinical Utility 0.408552386067
3777 77658 Influence Marketing and the Customer Experience | IBM Big Data & Analytics Hub 0.40851433695
3778 60445 How Quants Rank Their Communication Skills - Burtch Works 0.408402666832
3779 35823 situational awareness | The Big Data Hub 0.408381248712
3780 35031 Perceptions and realities of predictive analytics | The Big Data Hub 0.408248290465
3781 841 Idiro Analytics | Experts in Social Network Analysis, Big Data, Advanced & Predictive Analytics 0.408248290464
3782 53442 ShinyStat™ - Display the Report - Login Area for your web stats. 0.408248290464
3783 7981 The People Analytics Market Heats Up With New Cloud Offerings 0.408248290464
3784 81549 Comments on: About 0.408248290464
3785 41899 Logi Analytics | Business Intelligence & Data Discovery 0.408248290464
3786 23383 Comments on: Predictive Analytics: Investing and Selecting Software Properly 0.408248290464
3787 

In [217]:
tmp = df[df['analytics_sim'] > 0.4]
docs = tmp['body']
aG = DistanceGenerator(tmp)
sign = aG.calculate_signature(docs)

In [218]:
df['analytics_sim'] =  df.key_words.map(lambda x: get_cosine(sign, x))

In [219]:
dm_dict = dict()
dm_dict['data'] = 0.5
dm_dict['mining'] = 0.5
df['dm_sim'] =  df.key_words.map(lambda x: get_cosine(dm_dict, x))
dm_sim = df['dm_sim'].values.argsort()[::-1]

In [220]:
idx = 0
for i in dm_sim:
    if df.iloc[i]['dm_sim'] > 0.60:
        title = df.iloc[i]['title']
        sim_val = df.iloc[i]['dm_sim']
        idx += 1
        print idx, i, title, sim_val

1 17112 Topics of Interest 0.954133623823
2 17129 DMIN'08 Topics 0.952927987247
3 17105 DMIN'09 Topics 0.951223284653
4 17118 Topics of Interest 0.946538570444
5 6708 Data Mining Practice - GM-RKB 0.935436289067
6 6706 Data Mining Discipline - GM-RKB 0.932699523402
7 26363 OSU awards data mining, business analytics certificates to May graduates | Spears School of Business 0.930472224066
8 6709 Data Mining System - GM-RKB 0.926813264335
9 57713 Data Mining VS. Statistics 0.924866259868
10 75392 Data Mining 0.923578099738
11 389 Data Science Today: How to Become a Data Mining Analyst - Blog 0.918062580598
12 8950 Estard Software :: Data Mining :: Data Mining Definition 0.915652985679
13 66970 IMMM 2014 0.913343953221
14 66977 IMMM 2015 0.913092253171
15 43172 IMMM 2012 0.912993618636
16 43176 IMMM 2013 0.912738071393
17 43169 IMMM 2011 0.912380665483
18 66996 IMMM 2016 0.909888762088
19 58666 ICDM Topics 0.909225396289
20 81616 Data Mining Consulting Services - Abbott Analytics: Data Min

173 46348 Data Mining: Failure to Launch [ Free Webinar: Mar 15, Apr 21 ] 0.766924734205
174 38851 KDnuggets News 06:n17, Sep 12, 2006 0.766688920092
175 71794 Data Mining Scientist 0.765360974084
176 9584 New Technology | Data Mining Technologies Inc 0.764395168106
177 38896 KDnuggets News 09:n15, Aug 12, 2009 0.763913054603
178 45513 Social Networks for Data Mining, Analytics, Web Mining, and Text Mining 0.763724756128
179 38881 KDnuggets News 07:n24, Dec 18, 2007 0.763577123992
180 75122 BigPMA 2015 0.762629940406
181 10109 10 new data mining books from IGI - AnalyticBridge 0.762305663952
182 43656 WekaMOOC: Data Mining with Weka, online course 0.762021125502
183 57898 Data Miners - About Gordon Linoff 0.761830592776
184 14650 AAAI Sponsored Workshops—KDD-98 0.761548658395
185 1607 Datamining Conference Call For Abstracts | Data Mining 2016|London | U.K | OMICS Group Conferences 0.761409953481
186 38830 KDnuggets News 05:n18, Sep 27, 2005 0.761323842239
187 44133 Companion Website f

392 10190 Data Mining Techniques in CRM: Inside Customer Segmentation - AnalyticBridge 0.697511943304
393 45699 Visual Data Mining with Item Explorer 0.697065432309
394 71760 Education in Analytics and Data Mining in Latin America 0.696897484967
395 10140 Book: Data Mining and Analysis - Fundamental Concepts and Algorithms - AnalyticBridge 0.696873147645
396 47046 KDnuggets News 2012:n27, Nov 28 Where did you apply analytics? Free software versions; Great webinar 0.696654226421
397 44502 KDnuggets™ News 13:n26, Oct 30 0.695963168317
398 83170 KDnuggets™ News 13:n12, May 8 0.695948211318
399 45473 Web Mining 0.695920205725
400 46388 KDnuggets News 2011:n25, Oct 26 Methods for Data Analysis/Mining? CrossValidated 0.695863330904
401 73504 chorian2 | SmartData Collective 0.695141335636
402 45927 Misic Data Mining and Discovery Solutions 0.694708802921
403 34002 Tokyo — Places - Japan City Open Data Census 0.694502504831
404 23530 Pintung 屏東縣 — Places - Your Open Data Census 0.694237017533


682 47233 New book: Data Mining Applications with R 0.659900504926
683 167 2015 SIGKDD Service Award | Sig KDD 0.659736158295
684 58042 Featured Posts - Data Plumbing 0.659672726931
685 39818 KDnuggets™ News 15:n30, Sep 16: 60+ Free Books on Big Data, Data Science; Analytics Jobs Length; Sal 0.659472632756
686 83165 KDnuggets™ News 13:n04, Feb 20 0.659433306253
687 43484 Data Mining / Analytic Courses, Events News, May 2013 0.65935069489
688 45658 What is the importance of Dark Data in Big Data world? 0.65931499428
689 82988 Webinar: Data Mining: Failure to Launch [Mar 20] 0.659301073014
690 58026 All Posts - Data Plumbing 0.65926470763
691 83166 KDnuggets™ News 13:n07, Mar 14 0.659093939965
692 83080 DMRadio: Best Practices for Data Mining, [recorded] 0.658904500904
693 43644 SFBayACM Mining Big Data Camp (free), Oct 12 0.658828595395
694 38863 KDnuggets News 07:n06, Mar 27, 2007 0.658807845868
695 47036 KDnuggets News 2012:n12, May 17 Poll: Analytics, Data Mining, Big Data Software U

1031 79435 Data Preparation: The Key to Unravelling the Big Data Opportunity - insideBIGDATA 0.633201771032
1032 73062 Managing Big Data? Make Sure You Know the Challenges | SmartData Collective 0.633175288195
1033 47199 Top KDnuggets tweets, Nov 1-3: Videos from 2013 Strata + Hadoop World; Top 10 data mining algorithms 0.633161873518
1034 57084 responsible-data - dgpblog 0.632975441972
1035 12475 Analytics (DSBDA 2015) 0.632900670115
1036 46461 MassMutual: Data Engineer 0.632889761767
1037 41093 Data Marts as an indispensable analytical tool 0.632790430407
1038 43725 Upcoming October Webcasts on Analytics, Big Data, Data Mining, Data Science 0.632666456012
1039 23099 Quality issues, measures of interestingness and evaluation of data mining models 0.632486766719
1040 26600 ICDM-05: 2005 IEEE International Conference on Data Mining 0.632455532034
1041 44300 2014 May Publications 0.632400828446
1042 75532 Jian Pei 0.632280409358
1043 83173 KDnuggets™ News 13:n17, July 17 0.632250885855
1

1353 3514 Data Mining Server 0.616296972618
1354 17989 Taxonomy of Data Scientists - Data Science Central 0.616231406911
1355 22681 Simply Salford Blog 0.616182180269
1356 45797 5 Not-to-be-Missed Ideas about Big Data 0.616115480094
1357 43473 Top news in March: Data Science Code of Conduct; Top LinkedIn Groups; Data Driven Journalism Tutoria 0.616083114976
1358 43257 Beagli: Finding value in your personal data 0.616067336976
1359 41244 How to Use Data Visualizations to Win Over Your Audience 0.616065105671
1360 16274 Data Science Central Weekly Digest - November 30 - Data Science Central 0.616033887968
1361 11805 Big Data Lessons from the NFL Draft - AnalyticBridge 0.616028423017
1362 44813 Edward Tufte 0.616022321607
1363 79318 2015 Survey of Data Scientists Reveals Strategic Insights - insideBIGDATA 0.615974657564
1364 83294 KDnuggets™ News 14:n03, Feb 5 0.615969547219
1365 18332 Data Intelligent Platform | ClearStory Data 0.615934841756
1366 17976 Treat Your Data Like an Asset – No

1692 18027 Weekly Digest - April 27 - Data Science Central 0.603857687996
1693 18053 Weekly Digest - May 11 - Data Science Central 0.603793325495
1694 33719 What is data quality all about and how to run a data cleaning project? | Algolytics – Analyze your d 0.603785526512
1695 45795 3 Things About Data Science You Won’t Find In Books 0.603770150942
1696 61584 Trillium Software for Big Data 0.603762997815
1697 74172 Data Scientist: The Sexiest Job of the 21st Century - HBR 0.603760079303
1698 41175 Exclusive Interview: Big Data and Data Science at UN 0.603735285641
1699 13818 Starting Small with Big Data - insideBIGDATA 0.603558011796
1700 4802 Prague — Places - Czech Republic Open Data Census 0.60355489109
1701 45779 Top LinkedIn Groups for Analytics, Big Data, Data Mining, and Data Science – from “Big Bang” to Now 0.603449944018
1702 273 Data Architecture News 0.60338578546
1703 44811 ecosystem 0.603365382323
1704 46735 Flip the 80/20 Rule for Analytics in the Hadoop Data Lake - Data 

In [221]:
tmp = df[df['dm_sim'] > 0.6]
docs = tmp['body']
aG = DistanceGenerator(tmp)
sign = aG.calculate_signature(docs)

In [222]:
df['dm_sim'] =  df.key_words.map(lambda x: get_cosine(sign, x))

In [223]:
ml_dict = dict()
ml_dict['machine'] = 0.5
ml_dict['learning'] = 0.5
df['ml_sim'] =  df.key_words.map(lambda x: get_cosine(ml_dict, x))
ml_sim = df['ml_sim'].values.argsort()[::-1]

In [224]:
idx = 0
for i in ml_sim:
    if df.iloc[i]['ml_sim'] > 0.30:
        title = df.iloc[i]['title']
        sim_val = df.iloc[i]['ml_sim']
        body = df.iloc[i]['body']
        idx += 1
        print idx, i, title, sim_val
        #print body

1 75554 Machine Learning 0.866469607418
2 71497 The Best Advice From Quora on ‘How to Learn Machine Learning’ 0.855138249618
3 6764 Machine Learning Subject Area - GM-RKB 0.852802865423
4 83401 Machine Learning Table of Elements Decoded 0.83607961715
5 83545 Model-Based Machine Learning, Free Early Book Draft 0.81569254668
6 71832 Uber ATC: Machine Learning Specialist 0.812427912759
7 71530 5 Best Machine Learning APIs for Data Science 0.80276910079
8 71795 Machine Learning Scientists 0.80171994379
9 80615 About - Machine Learning Mastery 0.80019875922
10 71679 Introducing Quora’s Machine Learning Sessions Series 0.79360293318
11 45888 Top 10 Machine Learning Videos on YouTube 0.789875493347
12 50027 Machine learning - Wikipedia, the free encyclopedia 0.788332445914
13 50028 Machine learning - Wikipedia, the free encyclopedia 0.787175581668
14 11418 Learning Path : Your mentor to become a machine learning expert 0.77897713147
15 72312 Machine Learning Department - Carnegie Mellon Unive

155 83283 Geoffrey Hinton talks about Deep Learning, Google and Everything 0.617822993478
156 30621 throwdown | The Official Blog of BigML.com 0.617758400749
157 4950 Comments on: How NoSQL Has Fundamentally Changed Machine Learning 0.617213399848
158 71526 H2O World 2015 – Day 3 Highlights 0.616324359304
159 71640 KDnuggets™ News 15:n37, Nov 11: 5 Best Machine Learning APIs; Beginners Guide: Apache Spark Machine  0.616072021772
160 2533 machine learning 0.615384615385
161 71861 Boeing: Machine Learning Technologist – Level 4/5 0.615339289225
162 62600 MLJ: Machine Learning for Science and Society 0.614875461902
163 45728 50 Useful Machine Learning & Prediction APIs 0.613971500994
164 31609 Machine Learning with Python at PyConES 2015 | The Official Blog of BigML.com 0.61388783627
165 68764 RE•WORK | Blog 0.61378996638
166 31606 The Future Impact of Machine Learning | The Official Blog of BigML.com 0.612637474633
167 39842 Scikit-learn and Python Stack Tutorials: Introduction, Implemen

334 40043 Machine Learning Engineer 0.551463364246
335 78767 Data Science 101: An Introduction to scikit-learn - Machine Learning in Python - insideBIGDATA 0.55135136424
336 68755 RE•WORK | Blog - Deep Learning with Google Brain's Oriol Vinyals 0.550612131519
337 31585 Machine Learning Made (Too) Easy | The Official Blog of BigML.com 0.550253959968
338 12474 Big Data Cloud 0.549506259785
339 58648 Machine Learning textbook examples 0.549442255795
340 25012 PASCAL Bootcamp in Machine Learning, Vilanova 2007 - VideoLectures - VideoLectures.NET 0.549299340081
341 46032 BigML 0.549286391236
342 44141 Interview: Joseph Sirosh, Microsoft on Azure ML and the Emerging Data Science Economy 0.549154174094
343 86053 Manning | Machine Learning in Action 0.548762302957
344 45754 20 Lessons From Building Machine Learning Systems 0.548617330231
345 39861 PhD position on Machine Learning for Gait Analysis 0.547774334463
346 63546 Zack Chase Lipton's Home Page - Music and Machine Learning 0.54772255750

550 72060 Andrej Karpathy 0.503852946645
551 41097 H2O Deep Learning Webinar 0.503578083893
552 31680 anomaly detection | The Official Blog of BigML.com 0.503550782789
553 44641 Learning and Teaching Machine Learning: A Personal Journey 0.503456369967
554 53649 Contact -Skymind 0.503435077775
555 83005 Cartoon: Mother Of All Data 0.503391638153
556 83035 The Age of Big Data – BBC Documentary 0.503391638151
557 31659 API | The Official Blog of BigML.com 0.503290078787
558 17839 All Machine Learning Models Have Flaws - Data Science Central 0.503241120949
559 43965 ibai-publishing 0.502942438179
560 58643 Peter Flach 0.502871579684
561 82999 KDnuggets Cartoon looks at IRS, Big Data, and Taxes 0.502865352971
562 45319 Sibyl 0.502570711032
563 83632 Strata + Hadoop World, New York City, Sep 29 – Oct 1 0.501964644111
564 46071 Caffe 0.501848435139
565 72313 People-Machine Learning Department - Carnegie Mellon University 0.501745206004
566 17106 DMIN'09 Programme 0.501363582548
567 79406 Usin

849 75645 Python 0.459859840435
850 46508 Booking: Data Scientist – Machine Learning 0.45977770811
851 40023 Senior Data Scientist 0.459398509936
852 71566 7 Essential Resources & Tips To Get Started With Data Science 0.45933498936
853 43431 Inside Deep Learning: Computer Vision With Convolutional Neural Networks 0.459334801217
854 45524 Kernel Machines — Kernel Machines 0.459167239076
855 43429 Watson Developer Cloud-Visual Recognition 0.458902931252
856 71468 Rich Data Summit, San Francisco, Oct 14 – early bird by Sep 14 0.458663858066
857 71649 Webinar: Text Mining Along the Drug Development Pipeline, Jan 28 0.458532392337
858 71829 Autodesk: Senior Data Analyst 0.458532392337
859 46557 Microsoft: Data Scientists in Redmond, Boston or Mountain View 0.458158177123
860 46591 Microsoft: Data Scientists in Redmond, Boston or Mountain View.. 0.458158177123
861 16214 Clojure Machine Learning & Statistics Libraries - Data Science Central 0.457824442599
862 34815 Machine Learning and Knowle

1148 71872 U. of Iowa Tippie College of Business: Business Director, Analytics Academy 0.424808594
1149 75757 Udacity 0.424767291269
1150 6156 2006 0.424363274061
1151 33676 Artificial Intelligence, Università di Firenze 0.424334212396
1152 44511 Cartoon: Watson and Artificial vs Natural Intelligence 0.424176438004
1153 40092 Globys: Applied Data Scientist, Machine Learning and Data Science 0.423843935846
1154 74366 A Visual Introduction to Machine Learning 0.423694985025
1155 68821 RE•WORK 0.423324390773
1156 31614 PAPIs Connect 2016 – Call for Proposals on track! | The Official Blog of BigML.com 0.423188593928
1157 83326 Interview: Anthony Bak, Ayasdi on Novel Insights using Topological Summaries 0.42308720997
1158 37141 About PASCAL | PASCAL 2 0.423076923077
1159 53660 Skytree Products 0.423075836529
1160 83458 Top stories for Mar 29 – Apr 4: Deep Learning, Dimensionality, and Autoencoders; The Grammar of Data 0.423014392746
1161 75668 Robert Tibshirani 0.422987632022
1162 14839 Big

1483 83646 Webinar On-Demand: Your First Hire in Predictive Analytics (Hint: it is not a Data Scientist) 0.387578709894
1484 83220 Top stories for May 4-10 0.38754557265
1485 78821 CrowdFlower Combines the Best of Machine and Human Intelligence with New AI Product - insideBIGDATA 0.387399232977
1486 71573 KDD 2016 San Francisco Call For Papers | Workshops | Tutorials | KDD Cup 0.38733835098
1487 19251 deep learning - Data Plumbing 0.387298334621
1488 15028 HPC Advisory Council Archives - insideBIGDATA 0.387298334621
1489 83639 Top KDnuggets tweets, Jun 30 – Jul 06: Click Testing Proved that Beards Are Still A Thing; 16 Free # 0.387218339095
1490 71491 Crushed it! Landing a data science job 0.387033851068
1491 45916 Libraries and Development Kits for Data Mining 0.386632300878
1492 75706 Software Engineer 0.386587666529
1493 31601 BigML Acquires Best-of-Class Association Discovery Technology ‘Magnum Opus’, Expands Product Offerin 0.386397789193
1494 43228 Enter a KDD Cup or Kaggle Compe

1785 71823 Discovery Communications: Manager – Data Science 0.358315891731
1786 47147 PAW Predictive Analytics World April 2013 San Francisco 0.35828718195
1787 74641 Machine-Learning Solutions for Government | Skytree 0.358248860416
1788 43236 Top stories for Jan 11-17: Research Leaders on Data Mining/Big Data key trends, top papers; Deep Lea 0.358172254223
1789 75737 Text Classification 0.358152544011
1790 71598 Tamr 2016 Data Management Predictions 0.358019505197
1791 31594 How to Perform Clustering in a Single Command Line | The Official Blog of BigML.com 0.357942866494
1792 83136 KDnuggets 13:n29, Thanksgiving and Big Data Cartoon; Harvard Data Science Course 0.357924728358
1793 40031 Applied Data Scientist (Post-Doctoral) 0.357888347741
1794 15377 PAPIs '15, 6th-7th August 2015 | Lanyrd 0.35783904271
1795 83669 Webinar: Data Mining: Failure to Launch[Aug 18] 0.357735400268
1796 15526 Optimal Thresholding of Classifiers to Maximize F1 Measure - Springer 0.357713646795
1797 44617 T

2107 14925 Data Science 101 Archives - insideBIGDATA 0.330896227226
2108 44874 Genetic 0.330862051269
2109 45907 New Book: Mining Latent Entity Structures 0.330836419275
2110 79472 Andrej Karpathy Academic Website 0.330794862653
2111 21825 Ask a Data Scientist: Unsupervised Learning - insideBIGDATA 0.330787733209
2112 79916 scikit-learn Tutorials — scikit-learn 0.18.1 documentation 0.330770600828
2113 45776 Text By the Bay conference, San Francisco, Apr 24-25 0.330686888682
2114 32495 UCI Machine Learning Repository: Audiology (Original) Data Set 0.330618032657
2115 43525 Process Mining Movie 0.330473117555
2116 46515 Boeing: Advanced Information Technologist Text Analytics 0.330414310187
2117 25932 Documentation scikit-learn: machine learning in Python — scikit-learn 0.18.dev0 documentation 0.330409300227
2118 44991 Joseph Sirosh 0.330400394196
2119 41217 2015 Sep Top Tweets 0.330397027265
2120 50693 Power Java by Mark Watson [Leanpub PDF/iPad/Kindle] 0.330388434751
2121 46450 Salford

2399 43553 KDD 2013 Industry Practice Expo 0.309628107926
2400 43411 What do you want to learn? Big Data TechCon How-To Conference, Apr 26-28, Boston 0.309589058628
2401 15118 Blog 0.309523809524
2402 60139 Ben Scully's Page - Data Science Central 0.309449156809
2403 71482 How Big Data Helps Build Smart Cities 0.309399172755
2404 86103 Deep Learning Summit Boston 0.309334976335
2405 71533 2016 Conferences on Big Data, Predictive Analytics 0.309326352589
2406 43253 Active Data Mining, Data Science blogs 0.309232503814
2407 45161 Oxford 0.309220843218
2408 44515 Data Science for Social Good, Chicago Jun-Aug 2014, Apply by Feb 1 0.309032658305
2409 17826 A Visual Introduction to Machine Learning - Data Science Central 0.309015721575
2410 83495 NYC Data Science Academy 12 weeks Bootcamp, Apply by May 15 0.30891331371
2411 78331 AnalyticTalent.com 0.308824319795
2412 44507 AltaPlana 2014 Text Analytics Market Study 0.308823529412
2413 45628 Bot or Not: an end-to-end data analysis in Python 

In [225]:
tmp = df[df['ml_sim'] > 0.3]
docs = tmp['body']
aG = DistanceGenerator(tmp)
sign = aG.calculate_signature(docs)

In [226]:
df['ml_sim'] =  df.key_words.map(lambda x: get_cosine(sign, x))

In [227]:
dsj_dict = dict()
dsj_dict['data'] = 0.3
dsj_dict['science'] = 0.3
dsj_dict['jobs'] = 0.3
df['dsj_sim'] =  df.key_words.map(lambda x: get_cosine(dsj_dict, x))
dsj_sim = df['dsj_sim'].values.argsort()[::-1]

In [228]:
idx = 0
for i in dsj_sim:
    if df.iloc[i]['dsj_sim'] > 0.55:
        title = df.iloc[i]['title']
        sim_val = df.iloc[i]['dsj_sim']
        idx += 1
        print idx, i, title, sim_val

1 66569 Weekly Digest - April 28 - Data Science Central 0.776430975421
2 66578 Weekly Digest - December 1 - Data Science Central 0.764758808669
3 16476 How the gap between data science and statistics grew over time - Data Science Central 0.762917334563
4 66505 The ABCD's of Business Optimization - Data Science Central 0.762539377585
5 56716 Data Science and Business Analytics Degree Program 0.760885910253
6 60044 Awesome Data Science Repository - Data Science Central 0.759942816455
7 66619 Weekly Digest - November 10 - Data Science Central 0.755382390779
8 6783 Data Science Archives - Galvanize 0.752981182662
9 17986 Two periodic tables for data scientists - Data Science Central 0.75271433516
10 18041 Weekly Digest, January 5 - Data Science Central 0.751943759833
11 60029 Update about the Data Science Apprenticeship - Data Science Central 0.750912941748
12 66615 Weekly Digest - May 19 - Data Science Central 0.75025530782
13 16225 Columbia data science course, week 1: what is data scien

169 17950 Tools to automate... more time to innovate - Data Science Central 0.708737719556
170 17722 25 Data Scientists Popular on LinkedIn - Data Science Central 0.708470793747
171 58100 Data Science Patterns: Preparing Data for Agile Data Science - Data Science Central 0.70823688704
172 33131 About the NYU Center for Data Science - NYU Center for Data Science 0.708231577022
173 62734 Data Science: Career Outlook - WPI 0.708027548028
174 60028 Update about our data science apprenticeship - March 29, 2014 - Data Science Central 0.707872877599
175 57650 Unstructured Data: InfoGraphics - Big Data News 0.707838397297
176 66660 Zipfian Academy versus Data Science Apprenticeship - Data Science Central 0.707817404507
177 66324 Data Science is Dead - Long Live the Data Scientist - Data Science Central 0.707609868716
178 66600 Weekly Digest - July 28 - Data Science Central 0.70741282179
179 49826 What Do You Expect from Inside Your Data? - Data Science Central 0.707407613696
180 82122 Data Sci

395 12025 The top 10 worst graphs - AnalyticBridge 0.683506742601
396 58195 Spectacular Analysis of 2,719 Twitter Users Talking about "Data Scientist" - Data Science Central 0.683341863394
397 33147 Partnerships - NYU Center for Data Science 0.683329301484
398 58772 The Science of Segmentation: What Questions Should You Be Asking Your Data? - Data Science Central 0.683266718076
399 18046 Weekly Digest, June 15 - Data Science Central 0.683213456235
400 19244 Great IoT, Sensor and other Data Sets Repositories - Data Plumbing 0.683000737722
401 60246 Harish Moodalbail's Page - Data Science Central 0.682863359642
402 28461 Data Science at the Command Line - Data Science Central 0.682793885675
403 5048 Last chapter of my data science book completed - Data Science Central 0.682775203423
404 28502 I-COM Data Science Hackathon 2015 - Data Science Central 0.682721676879
405 45645 The different data science roles in the industry 0.682694447141
406 60031 Discussion Forum - Data Science Central 0.

686 66250 A Plethora of Open Data Repositories (i.e., thousands!) - Data Science Central 0.661968463895
687 66626 Weekly Digest, October 12 - Data Science Central 0.661960985576
688 71487 Data Science Education: Where It Needs To Begin 0.661875721113
689 28636 Data to explain increasing revenue gap? - Data Science Central 0.661842240242
690 5253 Tell us: as a data scientist, what is your super power? - Data Science Central 0.661782596008
691 16332 Developer Vs Coder - Data Science Central 0.661722721533
692 17706 This page has been moved - Data Science Central 0.661722721533
693 18128 You Will Meet Too Much False Precision - Data Science Central 0.661693159885
694 46176 Data Science Platform 0.661676601618
695 19573 Joaquim Coll's Page - Data Science Central 0.661448504642
696 17581 Susan Scrupski's Page - Data Science Central 0.661437827766
697 28627 Data Science Bootcamps, Anyone Ever Attend One? - Data Science Central 0.661300071266
698 19827 Saquib Kothawala's Page - Data Science C

1030 60393 Niraj Kumar's Page - Data Science Central 0.646827498274
1031 5234 Statistical Modeling steps - Data Science Central 0.646827498273
1032 42157 Data Science MSc at Lancaster University 0.646818612085
1033 5302 The Data Science Ecosystem in One Tidy Infographic - Data Science Central 0.646761666764
1034 18157 DSC Webinar Series: Using Data Science Techniques for Automatic Clustering of IT Infrastructure Aler 0.646689787584
1035 44178 2015 Predictions – What’s Next for Data Scientists? 0.646652305324
1036 58771 The 5 Most Influential Visualizations of All Time - Data Science Central 0.646632301493
1037 5349 The top 1% data users consume 99% of all the data being produced. True or false? - Data Science Cent 0.646554550588
1038 33146 Professor Claudio Silva Voted Chair in Prestigious IEEE Committee - NYU Center for Data Science 0.646508183835
1039 5117 Nine new, great articles and resources posted externally - Data Science Central 0.646507517606
1040 15715 DSC Data Science Search

...

3094 46851 The Top 10 Business Intelligence Trends for 2016 - Data Science Central 0.590839156702
3095 28665 How to find the real web domain hidden behind a bit.ly shortened URL? - Data Science Central 0.590790839829
3096 33149 Text as Data Speaker Series - NYU Center for Data Science 0.590777019637
3097 66074 Sasmita Samantaray's Page - Data Science Central 0.590739488912
3098 11779 AnalyticBridge announcement - AnalyticBridge 0.590734337507
3099 19540 Ihe Onwuka's Page - Data Science Central 0.590717348824
3100 44822 Entrepreneur 0.590677541887
3101 5273 The 7 Most Data-Rich Companies In The World - Data Science Central 0.590651252101
3102 869 Directors Letter - Master in Data Science - Barcelona GSE 0.590624423219
3103 28508 Intensive Longitudinal Methods - Data Science Central 0.590624423219
3104 60045 Correlation and R-Squared for Big Data - Data Science Central 0.590481397566
3105 17598 Thomas J Bate's Page - Data Science Central 0.590470228063
3106 44153 Panel on Data Science Ed

3453 33134 Required Courses - NYU Center for Data Science 0.580458661931
3454 46047 Booz Allen 0.580442102473
3455 47200 Top KDnuggets tweets, Nov 11-12: 5 Fundamental Concepts of Data Science; 11 TED talks explore the da 0.580421732867
3456 58095 DATA MINING: FAILURE TO LAUNCH - How to Get Predictive Modeling Off the Ground and Into Orbit - Data 0.580411127057
3457 41211 Top Datapreneurs in data science 0.580407942841
3458 28634 Data Scientist with only BS (non CS, math/statistics) - Data Science Central 0.580376112731
3459 19612 Kenneth C Black's Page - Data Science Central 0.580372538675
3460 17905 Workshop: Big Data Could Spell Big Legal Troubles - Data Science Central 0.580370430699
3461 28507 Integrating Spark and R (Free Webinar) - Data Science Central 0.580362926158
3462 82986 Data Mining / Analytic Publications News, Feb 2013 0.58035560417
3463 49821 Unified Secure On-/Off-line Analytics - Data Science Central 0.580318670603
3464 1862 Data Science News Daily 0.580318670603
346

3807 16309 Data Scientists Must Focus on Data Security Risk - Cyber Risk Report 2015 - Data Science Central 0.57131198904
3808 44430 2014 Sep Courses, Education 0.571297507455
3809 47065 GEQuest: 2 Big Data Science Quests on Kaggle 0.571214114958
3810 83136 KDnuggets 13:n29, Thanksgiving and Big Data Cartoon; Harvard Data Science Course 0.571204801279
3811 28488 Getting Started with R and Data Analysis - Data Science Central 0.57118577194
3812 28495 Hadoop Deployment Best Practices: Scalability, Robustness, Flexibility - Data Science Central 0.571099328621
3813 10234 Two interesting books - AnalyticBridge 0.57105743529
3814 11888 Great Friday Reading - AnalyticBridge 0.57105743529
3815 43504 Top news, jobs in April: Top LinkedIn Groups for Analytics, Big Data; Learning from “Learning from D 0.570982119904
3816 41266 2015 Oct News, Features 0.57094419789
3817 45792 2015 May 0.570929576248
3818 57530 Alexandra English's Page - Big Data News 0.570911217258
3819 5139 Phenomenalism and Unex

4161 5160 Privacy, Personalization, and the IOT - Retail - Data Science Central 0.562351594858
4162 32264 Brasília — Places - Censo de Dados dos Municípios do Brasil 0.562308699454
4163 15744 An extensive glossary of big data terminology - Big Data News 0.562269471817
4164 4848 Automation Is the New Reality for Big Data Initiatives 0.562229348844
4165 17956 Top 16 articles for week ending October 24 - Data Science Central 0.562214183413
4166 75405 Data Scientist 0.562196384044
4167 11691 Venkatesh Chellappa's Page - AnalyticBridge 0.562134146302
4168 5005 Inferential Modeling and Application of Analogs - Data Science Central 0.562013077905
4169 16474 How semantics is creating a better internet - Data Science Central 0.562002664172
4170 66123 Tullio Siragusa's Page - Data Science Central 0.561992455963
4171 16368 Ensuring Success of Early Modeling Initiatives - Data Science Central 0.561963559335
4172 58105 Past Events - Data Science Central 0.561945090779
4173 66532 From Data Analyst t

4482 66391 How is Big Data Changing the World? - Data Science Central 0.55375186129
4483 80884 27 free data mining books - AnalyticBridge 0.55375186129
4484 75385 Data Architect 0.553610238206
4485 46846 The Online M.S. in Health Informatics at St. Scholastica - Data Science Central 0.553565838808
4486 45500 Zementis 0.553557661137
4487 28613 Can data scientists replace business analysts? - Data Science Central 0.553539985841
4488 83219 Top stories for May 18-24 0.553509303462
4489 44569 Top stories in January: Tutorial: Data Science in Python; Learning from Data, Caltech free online co 0.553498663901
4490 58088 Big Data Bootcamp - December 7 to 9 2015, Tampa, FL - Data Science Central 0.553467778326
4491 5039 It's Sunny Skies For Big Data - Data Science Central 0.553461135603
4492 57586 50+ Open Source Tools for Big Data (See Anything Missing?) - Big Data News 0.553449461384
4493 75805 Cartoons 0.553411223326
4494 58200 Using phone call recordings for predictive modeling - Data Scienc

In [229]:
tmp = df[df['dsj_sim'] > 0.55]
docs = tmp['body']
aG = DistanceGenerator(tmp)
sign = aG.calculate_signature(docs)

In [230]:
df['dsj_sim'] =  df.key_words.map(lambda x: get_cosine(sign, x))

In [231]:
dsn_dict = dict()
dsn_dict['data'] = 0.3
dsn_dict['science'] = 0.3
dsn_dict['news'] = 0.3
df['dsn_sim'] =  df.key_words.map(lambda x: get_cosine(dsn_dict, x))
dsn_sim = df['dsn_sim'].values.argsort()[::-1]

In [232]:
idx = 0
for i in dsn_sim:
    if df.iloc[i]['dsn_sim'] > 0.60:
        title = df.iloc[i]['title']
        sim_val = df.iloc[i]['dsn_sim']
        idx += 1
        print idx, i, title, sim_val

1 57641 The data science alphabet - Big Data News 0.809029952165
2 81966 Data Science News | Data Science Institute 0.804180692896
3 75402 Data Science Jargon 0.796575403901
4 13218 Data Science News 23 June 2015 - My Data Mining Resources 0.796533674408
5 79579 What is Data Science? | Data Science Institute, U.Va. 0.791256568075
6 46175 Data Science Fellows 0.784665762704
7 14927 Data Science 101 Archives - Page 2 of 8 - insideBIGDATA 0.783789566329
8 81984 Data Science at NYU News - Data Science at NYU 0.783235934411
9 79148 What is Data Science? - NYU Center for Data Science 0.777593185921
10 57650 Unstructured Data: InfoGraphics - Big Data News 0.772187342505
11 14928 Data Science 101 Archives - Page 3 of 8 - insideBIGDATA 0.77141556065
12 79631 Programs & Admissions - NYU Center for Data Science 0.769112198421
13 57581 20 Big Data Repositories You Should Check Out - Big Data News 0.767590189676
14 79147 Careers in Data Science - NYU Center for Data Science 0.763828739597
15 15020 

175 41212 Top Datapreneurs in data science 0.715303358073
176 66612 Weekly digest, New Data Science Central Competition - Data Science Central 0.71511211612
177 57575 William Vorhies's Page - Big Data News 0.714879127003
178 23966 How To Stem The Global Shortage Of Data Scientists | TechCrunch 0.714749877804
179 16236 Could Data Science lead to societal harmony? - Data Science Central 0.714741689892
180 57487 Data Science Driven Approaches to Malware Detection - Big Data News 0.714741689892
181 57562 Michael Scola's Page - Big Data News 0.714580561923
182 5324 The multiple facets of data science - Data Science Central 0.714559907483
183 18063 Weekly Digest, October 26 - Data Science Central 0.714485096205
184 57541 Courtney Klepsch's Page - Big Data News 0.713932893462
185 10217 New book: Doing Data Science - Straight Talk from the Frontline - AnalyticBridge 0.713854775163
186 82265 Education / Training Archives - Page 2 of 17 - insideBIGDATA 0.713771885812
187 1863 Data Science Dojo |

393 17898 Big data and the retail industry: infographics - Data Science Central 0.690803083672
394 45963 Analytics Vidhya 0.690665355683
395 83857 Yet Another Data Blog: Data Science Bootcamp Programs - Full TIme, Part Time and Online 0.690588463881
396 17817 A Simple Introduction to Data Science - Data Science Central 0.690310104466
397 17776 6 stages of debugging - Data Science Central 0.69011648481
398 10231 Three data science journals - AnalyticBridge 0.690065559343
399 16292 Data Science Test - How do you rank? - Data Science Central 0.689977609954
400 377 Press 0.689969309177
401 66642 What is Hadoop? Great Infographics Explains How it Works - Data Science Central 0.689861970148
402 47217 Alpine Data Labs 2014 Predictions 0.689645493489
403 66590 Weekly Digest - February 9 - Data Science Central 0.689603925512
404 16320 Data Visualization: A Day in the Life of Worldreader - Data Science Central 0.689595385473
405 66584 Weekly Digest - December 8 - Data Science Central 0.689584113

689 82261 Google News Feed Archives - insideBIGDATA 0.671918388959
690 66041 Ratikant Pratapsingh's Page - Data Science Central 0.671775298207
691 58763 How Data Science is Preventing College Dropouts and Advancing Student Success - Data Science Central 0.67171661328
692 5285 Data Science has a Terminology Problem - Data Science Central 0.671674056357
693 66408 Infographic by Data Camp - Data Science Central 0.671660494259
694 44541 Free Tutorial: Data Science in Python 0.671634284312
695 17674 umamahesh's Page - Data Science Central 0.67150595336
696 44444 MOOC: “Process Mining: Data science in Action” 0.671466344669
697 66552 Two big datasets to challenge your data science expertise - Data Science Central 0.671465295102
698 17700 15 great big data articles from top news outlets - Data Science Central 0.671459629703
699 66630 Weekly Digest, October 6 - Data Science Central 0.67145826717
700 17771 50 free copies of data science book, signed by the author: get yours! - Data Science Cent

1045 18040 Weekly digest - January 13 - Data Science Central 0.657031033626
1046 19609 Ken Gross's Page - Data Science Central 0.657030507809
1047 19476 Dwayne Pindling's Page - Data Science Central 0.656984789078
1048 17883 14 Great Machine Learning, Data Science, R , DataViz Cheat Sheets - Data Science Central 0.656940180913
1049 4980 How to identify the right data scientist for your company - Data Science Central 0.656939228591
1050 66072 Sarah Aryee's Page - Data Science Central 0.656811480271
1051 43672 Data Elite: Andreessen Horowitz Big Data incubator 0.656784710429
1052 28675 Julia for data science applications - Data Science Central 0.656778086154
1053 1748 Academics | Data Science Institute 0.656758671866
1054 16397 Forget Data Scientists - Make Everyone Data Savvy - Data Science Central 0.656753360412
1055 44838 Feature Extraction 0.656736073329
1056 66627 Weekly Digest, October 19 - Data Science Central 0.656734391984
1057 43719 Top KDnuggets tweets, Oct 28-29: The Mathemat

...

2072 58214 Dilip MS's Friends - Data Science Central 0.631142019646
2073 58213 David Schwartz's Friends - Data Science Central 0.631142019646
2074 58240 Renato Azevedo Sant Anna's Friends - Data Science Central 0.631142019646
2075 75636 Presentation 0.631101104148
2076 16167 Big Data: What Are The Key Jobs and Salaries Available? - Data Science Central 0.631073793962
2077 62734 Data Science: Career Outlook - WPI 0.631068031938
2078 19790 Riya Saxena's Page - Data Science Central 0.631059358253
2079 19427 Chris pilling's Page - Data Science Central 0.63103150284
2080 83219 Top stories for May 18-24 0.631000605947
2081 66470 R Packages: A Healthcare Application - Data Science Central 0.63099657846
2082 83588 2015 Jul Courses, Education 0.6309898162
2083 44636 KDnuggets Interview: Juan Miguel Lavista, Microsoft Data Science Team 0.630965916429
2084 57627 My thoughts on big data and data science: no, it's not hype - Big Data News 0.630964375012
2085 57578 suresh kumar gorakala's Page - Big

2423 17902 Big Data, Big Relevance - Data Science Central 0.624443357356
2424 17669 selcuk topal's Page - Data Science Central 0.624422396902
2425 66037 Ramin Mehrani's Page - Data Science Central 0.624414422016
2426 66095 Subudhi Prem's Page - Data Science Central 0.624369806679
2427 66490 So You Want to be a Data Scientist - Data Science Central 0.624349560277
2428 46941 Able Kuriakose's Page - Data Science Central 0.624340131631
2429 17927 Big Data Skills In Demand (Comprehensive List) - Data Science Central 0.624259746093
2430 46711 Data scientists don't scale. Get the full scoop - Data Science Central 0.624259496442
2431 45360 Starbucks 0.624255890659
2432 46779 Learn How to be Data Driven - Data Science Central 0.624255890659
2433 44343 Top KDnuggets tweets, Jun 9-10: Numeric Matrix Manipulation: Cheat Sheet; The First Law of Data Scie 0.6242501497
2434 19633 Linda Terlouw's Page - Data Science Central 0.624207579142
2435 83234 Is “Data Scientist” more than “Data Analyst” ? 0.624

2769 66146 Zach Butler's Page - Data Science Central 0.617366005463
2770 17764 5 Steps from Business Analyst to Data Scientist - Data Science Central 0.617360768288
2771 44646 SAS Competition for Top Data Scientist in UK, Ireland 0.617351073385
2772 44263 iMath Cloud Data Science Platform beta 0.617296756376
2773 44503 KDnuggets™ News 13:n30, Dec 11 0.617288390845
2774 57604 Big Data: The Amazing Numbers in 2015 - Big Data News 0.617278229784
2775 46681 Building a Data Culture: The Key to Success - Data Science Central 0.61718119881
2776 19477 Dyke Hensen's Page - Data Science Central 0.6171609848
2777 83122 Harvard CS109 Data Science Course, Resources Free and Online 0.617113454354
2778 66061 Ryan Waite's Page - Data Science Central 0.617060907345
2779 19650 Madhav cv's Page - Data Science Central 0.617060907345
2780 19709 Nagaraj Kulkarni's Page - Data Science Central 0.617031304137
2781 5226 Social Media for App Capture and Question Answering in Open Vocabulary Executable English ov

3075 75621 ParseHub 0.610816213521
3076 60309 Kevin Wang's Page - Data Science Central 0.610771556602
3077 46731 Event Wrap Up: A New Approach to Big Data - Pivotal - Data Science Central 0.610756543957
3078 70591 Badges & Players - Data Science Central 0.610586221578
3079 16178 Bringing Silicon Valley to the ICU: My Vision of Data for Good - Data Science Central 0.610586221578
3080 19641 Louis Giokas's Page - Data Science Central 0.610581637199
3081 60084 Alex Esterkin's Page - Data Science Central 0.610578011187
3082 58774 Your choice of Deep Net - Ep. 4 (Deep Learning SIMPLIFIED) - Data Science Central 0.610574123934
3083 46763 Infographic - Predictive Analytics World in color - Data Science Central 0.610571654182
3084 66081 Shin-ichi Kawamura's Page - Data Science Central 0.610560142774
3085 19537 Ian Thurman's Page - Data Science Central 0.610560142773
3086 76215 University of Wisconsin Data Science Master's Degree - Google+ 0.610560142773
3087 46874 Visualization for Data Managem

3435 1808 Blog 0.603022689156
3436 66283 Big Data, IOT and Security - OH MY! - Data Science Central 0.603022689156
3437 46558 Microsoft: R Data Scientist. 0.603014815101
3438 46662 Big Data, Hadoop, Data Science Courses: Big Discount Ends 12/22 - Data Science Central 0.603011273352
3439 16334 Difference between Machine Learning and Statistics - Data Science Central 0.603010281154
3440 46653 BI/Analytics Training at TDWI Las Vegas - Data Science Central 0.603007865406
3441 19732 Pamela Lee's Page - Data Science Central 0.602994168679
3442 45465 Victor Lavrenko 0.602994168679
3443 19558 Janardhanan PS's Page - Data Science Central 0.6029900995
3444 4997 Illegal drugs: do they really work? - Data Science Central 0.602927866812
3445 45116 Munich Re 0.602882890223
3446 17649 kary bheemaiah's Page - Data Science Central 0.602830065649
3447 60241 Gregory Piatetsky's Page - Data Science Central 0.602796286786
3448 71983 Should Data Science Include Ethics Training? 0.60276905184
3449 21544 Data

In [233]:
tmp = df[df['dsn_sim'] > 0.60]
docs = tmp['body']
aG = DistanceGenerator(tmp)
sign = aG.calculate_signature(docs)

In [234]:
df['dsn_sim'] =  df.key_words.map(lambda x: get_cosine(sign, x))

In [236]:
ba_dict = dict()
ba_dict['big'] = 0.5
ba_dict['data'] = 0.5
df['bd_sim'] =  df.key_words.map(lambda x: get_cosine(ba_dict, x))
ba_sim = df['bd_sim'].values.argsort()[::-1]

In [249]:
idx = 0
for i in ba_sim:
    if df.iloc[i]['bd_sim'] > 0.80:
        title = df.iloc[i]['title']
        sim_val = df.iloc[i]['bd_sim']
        idx += 1
        print idx, i, title, sim_val

1 24886 IBM - What is big data? 0.919592837188
2 57596 Big Data, a promised land where the Big Bucks grow - Big Data News 0.930782016945
3 8281 Big Data Strategy & Planning - For Dummies 0.921313047509
4 44017 IBM – Big Data – Big Data Conversations 0.935784092985
5 15687 IBM – Big Data – Big Data Conversations 0.935713391899
6 12435 Saving Time & Money with Big Data | Big Data Forum 0.89741977702
7 58789 72 Infographics about big data - DataViz 0.94893060796
8 16513 big data - DataViz 0.935521908324
9 15686 IBM Big Data – What is Big Data – United States 0.930125015689
10 11917 Important Thoughts For Start Ups Looking At Big Data - AnalyticBridge 0.906623008739
11 8271 Big Data - For Dummies 0.919093117107
12 25985 The International Conference on Big Data, Internet of Things, and Zero-Size Intelligence BDIZSI2015  0.887465022342
13 67028 IBM Big Data – What is Big Data – United States 0.934036598866
14 17794 72 Infographics about big data - Data Science Central 0.949871763084
15 36042

173 30516 Simply Improving Decision-Making | The Big Data Hub 0.913422337033
174 44472 What is Big Data – answers from thought leaders 0.90744850115
175 57651 Using Big Data to Target In-Market Shoppers - Big Data News 0.900125959551
176 78064 #IBMDataHero | IBM Big Data & Analytics Hub 0.926141738166
177 54183 Infographic: Big Data Exploration | The Big Data Hub 0.932038700427
178 30243 Do digital business, big data and the Internet of Things play well together? | The Big Data Hub 0.923873429462
179 39109 2013 IEEE International Conference on Big Data  (IEEE BigData 2013) 0.814588792491
180 35689 ROI | The Big Data Hub 0.910401588636
181 65753 Datafloq - The One-Stop Shop for Big Data 0.843289629495
182 35552 zone architecture | The Big Data Hub 0.920309781483
183 36645 Big data for social good | The Big Data Hub 0.902978023285
184 36494 Big Data Implementation Best Practices | The Big Data Hub 0.941822916467
185 13650 New Research Shows Businesses are Investing Heavily in Big Data An

397 57370 IBM big data platform - Bringing big data to the Enterprise  0.887657803423
398 82446 Chief data officers are data and analytics | IBM Big Data & Analytics Hub 0.928369699683
399 24627 YouTube | What's The Big Data? 0.870329872498
400 15791 The Case for Big Data Adoption - Big Data News 0.903715371061
401 54271 IBM Big Data in a Minute: Spotting trends with big data | The Big Data Hub 0.900214706351
402 31319 big data – Foreground Analytics, Big Data Studio 0.863412200109
403 18080 What is Big Data, and why should you care? - Data Science Central 0.925501894044
404 36647 Big Data, Little Privacy? | The Big Data Hub 0.913964040093
405 54239 Are data artists essential for big data success? | The Big Data Hub 0.918383251265
406 67059 Nitesh Chawla | IBM Big Data & Analytics Hub 0.915121182066
407 13718 The Elephant in the Cloud - insideBIGDATA 0.883170350724
408 14812 NewVantage Releases 4th Annual Big Data Executive Survey 2016 Survey - Big Data has Achieved Mainstr 0.812310529

688 24566 Bubble? What bubble? | What's The Big Data? 0.853264765483
689 54267 Streams Developer Day, part 2: Geospatial toolkit | The Big Data Hub 0.915165108883
690 34994 IBM Big Data in a Minute: Real-time possibilities | The Big Data Hub 0.894530737248
691 14972 Big Data Services Archives - insideBIGDATA 0.829920584689
692 35882 Twitter chat | The Big Data Hub 0.853392853126
693 11857 Enhancing Customer experience with Big Data Analytics - AnalyticBridge 0.92353985654
694 46024 Big Data Journal 0.875690045561
695 15805 Ultranyx Big Data - Big Data News 0.903393566856
696 35793 stream computing | The Big Data Hub 0.866575296516
697 55880 All Posts | SmartData Collective 0.874181061748
698 55229 The White House Recently Completed a Study on Big Data Privacy: Do You Care? | SmartData Collective 0.858523311289
699 30332 Fighting Insurance Fraud with Big Data | The Big Data Hub 0.861277673601
700 21541 Big Data - Quality or Quantity?  - Inside-BigData 0.874946897945
701 30425 Lost in a 

1005 30339 Twitter chat: The Future of Retail | The Big Data Hub 0.867634711045
1006 19132 Government Archives - Datanami 0.80406058945
1007 55883 Europe's Big Data Market | SmartData Collective 0.850884703635
1008 4917 2015 December — Data-Magnum 0.822716002144
1009 57642 The Data Scientist: Elusive or Illusive? - Big Data News 0.895834003608
1010 54088 Hadoop & Spark | The Big Data Hub 0.833814067057
1011 78127 query optimization | IBM Big Data & Analytics Hub 0.866231750554
1012 55839 Big Data Ethics: Rearranging the Puzzlers | SmartData Collective 0.86467448249
1013 66447 Online Courses to Freshen Up Your Knowledge of Big Data - Data Science Central 0.855124476592
1014 31275 News | Big Data for Productivity Congress 2015 0.831873153789
1015 78096 performance | IBM Big Data & Analytics Hub 0.854372840987
1016 15176 Careers | Kognitio 0.819862930887
1017 75638 Privacy 0.856250675308
1018 78957 Who’s Afraid of the Big Bad Data Wolf? - Trillium Insights 0.845174489068
1019 55811 Unstru

...

3320 54582 artificial intelligence (AI) | The Big Data Hub 0.817344119827
3321 18057 Weekly Digest - May 5 (New Format) - Data Science Central 0.838803238575
3322 66620 Weekly Digest - November 17 - Data Science Central 0.846468156921
3323 54518 Epic performance powered by analytics | The Big Data Hub 0.80606264314
3324 47052 2013 06-June 0.826204240054
3325 42189 Data Drive Thru: Gregory Piatetsky-Shapiro | LatentView Analytics 0.812833278717
3326 77765 Unlocking business value through enterprise Hadoop adoption | IBM Big Data & Analytics Hub 0.807122369936
3327 66582 Weekly Digest - December 29 - Data Science Central 0.834540913365
3328 57641 The data science alphabet - Big Data News 0.822519994453
3329 28635 When a data glitch turns great data into worthless gibberish - Data Science Central 0.803789378146
3330 35609 date warehouse | The Big Data Hub 0.84551512305
3331 17865 Are You Concerned Over Systems Security? Cloud Systems Can Be Vulnerable. - Data Science Central 0.81481593945

3620 44390 Data is not a fad 0.835742683952
3621 19561 Jason Cohen's Page - Data Science Central 0.807826364991
3622 82664 White Papers and Research 0.826415902899
3623 55029 Overview of Text Mining 0.821656083025
3624 36550 Blogs | The Big Data Hub 0.810305509361
3625 39929 Datasets for Data Mining and Data Science 0.809972257348
3626 82304 Data Science Archives - Page 2 of 22 - insideBIGDATA 0.810772148025
3627 40056 Big Data Analyst 0.807004623409
3628 35904 Aberdeen Group | The Big Data Hub 0.813877090583
3629 54451 Contact Us | The Big Data Hub 0.804065200461
3630 19192 ScaleOut Software Archives - Datanami 0.800412340986
3631 83209 2014 Apr News, Features 0.805915962164
3632 5111 Weekly Digest - April 29 - Data Science Central 0.815733112692
3633 19528 Harshali Singh's Page - Data Science Central 0.808282516597
3634 18048 Weekly digest - June 9 - Data Science Central 0.832950796207
3635 19123 Visualization Archives - Datanami 0.821374444767
3636 58026 All Posts - Data Plumbing 0.

3896 5308 The end of the Data Scientist Bubble - Data Science Central 0.818098470808
3897 35824 situational awareness | The Big Data Hub 0.807233773553
3898 35743 Smarter Cities | The Big Data Hub 0.805834054379
3899 43560 New Poll: Interested in Analytics, Big Data, Data Mining, Data Science education? 0.806405102606
3900 16287 Data Science: Paving the Way For More Open, Interactive Data Analysis - Data Science Central 0.807137972414
3901 52750 Big Data Governance | SmartData Collective 0.821400615946
3902 5048 Last chapter of my data science book completed - Data Science Central 0.811276854604
3903 78050 unstructured data | IBM Big Data & Analytics Hub 0.817381355307
3904 41092 Data Hierarchy of Needs 0.811782873489
3905 46816 Resolve identity conflicts within your data - IBM Webinar - Data Science Central 0.814776139793
3906 66487 Six categories of Data Scientists - Data Science Central 0.806680948391
3907 46637 Accelerate, Automate and Enrich Data Prep - Data Science Central 0.8115

4051 51804 Streamlined Data Refinery | Pentaho 0.801912978763
4052 18026 Weekly digest - April 21 - Data Science Central 0.827712536553
4053 83183 2014 Jan: Analytics, Big Data, Data Mining and Data Science News 0.803514676083
4054 66353 Do You Really Need a Unicorn? - Data Science Central 0.800291285089
4055 16453 How Clean Is Your Data? - Data Science Central 0.805242602338
4056 46895 Wrangle big data with our M.S. in Data Science - Data Science Central 0.800610825466
4057 11748 13 great articles about analytics / data science from major news outlets - AnalyticBridge 0.825234823562
4058 49826 What Do You Expect from Inside Your Data? - Data Science Central 0.802501459775
4059 66658 "You need an algorithm, not a Data Scientist". Um...not quite - Data Science Central 0.813952850686
4060 49812 The lowest paid data scientist - Data Science Central 0.800352153734
4061 44494 KDnuggets™ News 13:n05, Feb 27 0.8179982819
4062 5325 9 Must-Have Skills You Need to Become a Data Scientist - Data 

4177 75384 Data Analytics 0.822434661531


In [250]:
tmp = df[df['bd_sim'] > 0.80]
docs = tmp['body']
aG = DistanceGenerator(tmp)
sign = aG.calculate_signature(docs)

In [251]:
df['bd_sim'] =  df.key_words.map(lambda x: get_cosine(sign, x))

In [252]:
ds_dict = dict()
ds_dict['data'] = 0.5
ds_dict['science'] = 0.5
df['ds_sim'] =  df.key_words.map(lambda x: get_cosine(ds_dict, x))
ds_sim = df['ds_sim'].values.argsort()[::-1]

In [253]:
idx = 0
for i in ds_sim:
    if df.iloc[i]['ds_sim'] > 0.70:
        title = df.iloc[i]['title']
        sim_val = df.iloc[i]['ds_sim']
        idx += 1
        print idx, i, title, sim_val

1 6783 Data Science Archives - Galvanize 0.92220984172
2 75399 Data Science 0.917986703679
3 79579 What is Data Science? | Data Science Institute, U.Va. 0.915249233499
4 16325 David Donoho reflects on "50 Years of Data Science" - Data Science Central 0.915231884492
5 57641 The data science alphabet - Big Data News 0.909637638428
6 66505 The ABCD's of Business Optimization - Data Science Central 0.909121956747
7 18087 What Is The Profession Of Data Science Really About Now And In The Future? - Data Science Central 0.908018103987
8 60044 Awesome Data Science Repository - Data Science Central 0.907467665329
9 5306 The Data Science Venn Diagram Revisited - Data Science Central 0.904703979067
10 16476 How the gap between data science and statistics grew over time - Data Science Central 0.90374371271
11 56716 Data Science and Business Analytics Degree Program 0.90365199156
12 79148 What is Data Science? - NYU Center for Data Science 0.903514637435
13 1826 Data Science Blog | University of Wi

175 5342 The Science News Cycle - Data Science Central 0.84979323433
176 5104 My Data Science Book - Table of Contents - Data Science Central 0.849660203865
177 81985 Data Science at NYU News - Data Science at NYU 0.849332799565
178 66213 2013 Data Science Salary Survey, by O'Reilly - Data Science Central 0.849265064603
179 66464 Proposal for an Apprenticeship in Data Science - Data Science Central 0.849191827894
180 51869 The School of Data Science 0.849189365702
181 1865 Data Science Dojo | Careers 0.849089513898
182 46175 Data Science Fellows 0.848896908955
183 1822 Comments for Data Science Bootcamps 0.848528137424
184 60024 Looks like the Data Science Apprenticeship should be interesting but intense - Data Science Central 0.84841071846
185 10098 Online Master of Science in Data Science - Lewis University - AnalyticBridge 0.848146703916
186 66349 Discover, Access, Distill: The Essence of Data Science - Data Science Central 0.848098676652
187 17937 Thoughts of the week - Data Scienc

398 18051 Weekly Digest - March 31 - Data Science Central 0.823560028824
399 79632 Admission Requirements - NYU Center for Data Science 0.823544659658
400 58741 Approaches to test Hypothesis - Data Science Central 0.823457462034
401 66316 Data Science and Technology Monthly - September 2015 - Data Science Central 0.823421718923
402 17675 vishy's Page - Data Science Central 0.823300836743
403 18140 Data Science Central Co-Founder's Live Interview on Clear Channel - Data Science Central 0.823290645296
404 58279 The Past, Present and Future of Data Science – A Live Roundtable - DSC Webinar - Data Science Centra 0.823004707009
405 66602 Weekly digest - June 16 - Data Science Central 0.822969145011
406 66041 Ratikant Pratapsingh's Page - Data Science Central 0.822753351207
407 58763 How Data Science is Preventing College Dropouts and Advancing Student Success - Data Science Central 0.822681477143
408 5285 Data Science has a Terminology Problem - Data Science Central 0.82262935577
409 66408 

651 66130 Vinsent Paramanantham's Page - Data Science Central 0.80491448238
652 46709 Data Lakes, Reservoirs, and Swamps: A Data Science and Engineering Perspective - Data Science Centra 0.804828191797
653 18012 Venn style data visualisation - Data Science Central 0.804719904059
654 17772 50+ Free Data Science Books - Data Science Central 0.804709127
655 19528 Harshali Singh's Page - Data Science Central 0.804695388778
656 18040 Weekly digest - January 13 - Data Science Central 0.804695388778
657 19609 Ken Gross's Page - Data Science Central 0.804694744786
658 17731 30 great articles and resources from top news outlets - updated weekly selection - Data Science Cent 0.804685577168
659 19476 Dwayne Pindling's Page - Data Science Central 0.804638751005
660 17883 14 Great Machine Learning, Data Science, R , DataViz Cheat Sheets - Data Science Central 0.804584117384
661 4980 How to identify the right data scientist for your company - Data Science Central 0.804582951033
662 66072 Sarah Aryee

993 46693 Chief Data Officer Summit: Connect with NASA, Ford, United Airlines & More - Data Science Central 0.788418759109
994 33140 Machine Learning Can Guess Your Age - NYU Center for Data Science 0.788416527987
995 28582 Webinar: Maximizing ROI (using State-of-the-Art Data Science Techniques) - Data Science Central 0.788240781368
996 66362 The Data Science Toolkit - taking your first steps towards becoming a Data Scientist - Data Science  0.788185341859
997 28486 Free Webinar: GPU Computing for Data Science - Data Science Central 0.788160360944
998 17762 5 Signs That You Are NOT a Data Scientist - Data Science Central 0.788061459842
999 17645 jeanna frawley's Page - Data Science Central 0.787978959289
1000 81984 Data Science at NYU News - Data Science at NYU 0.787967016302
1001 60310 Khurram's Page - Data Science Central 0.787950880861
1002 60328 Laurie Cook's Page - Data Science Central 0.787932616921
1003 66110 Thanh Vu's Page - Data Science Central 0.787932616921
1004 18145 DSC W

...

2318 28485 Free Webinar - Big Data Analytics for Non-Programmers - Data Science Central 0.744814970559
2319 46646 Analyzing the Analyzers: What kind of a data scientist are you? - Data Science Central 0.744784932169
2320 60253 Hieu Tran's Page - Data Science Central 0.744781956061
2321 80857 Data Science Career Alert - April 2 - AnalyticBridge 0.744704116914
2322 44530 2014 Jan: Features, News, Software: Analytics, Big Data, Data Mining, and Data Science 0.744611250957
2323 17758 5 Industries That Need Big Data - Data Science Central 0.744559145945
2324 66645 What is the most used feature in any business intelligence solution? It's the Export to Excel button 0.744513459613
2325 45033 Life Science 0.744484113145
2326 58163 Can any data structure be represented by one-dimensional arrays? - Data Science Central 0.744452764964
2327 46637 Accelerate, Automate and Enrich Data Prep - Data Science Central 0.744452764964
2328 17736 35 books on Data Visualization - Data Science Central 0.7444375

2664 19377 B. John Shrewsbury, Ph.D.'s Page - Data Science Central 0.732854877302
2665 46654 Big Data Analytics London 2015 - Complimentary Pass - Data Science Central 0.732845779863
2666 45605 SAS Adds Certifications for Big Data and Data Science 0.732844255538
2667 16212 Choropleth in D3.js and Pandas (iPython Notebook) - Data Science Central 0.732828108793
2668 60226 Evelina Gabasova's Page - Data Science Central 0.732797953119
2669 10169 Book: Practical Data Science with R - AnalyticBridge 0.732770722301
2670 43225 Research Leaders on Data Mining, Data Science, and Big Data key trends, top papers 0.73274621537
2671 15797 Weekly Digest - August 26 - Big Data News 0.732718039835
2672 5163 Proposal for a new type of scoring system - Data Science Central 0.732709181802
2673 57531 Amol Vijay Gothe's Page - Big Data News 0.732679228074
2674 16450 How big data can terrorize global terrorism - Data Science Central 0.732673062116
2675 18006 What if the Story Doesn't Match the Data? NYTimes 

3011 11724 pratj's Page - AnalyticBridge 0.720360270225
3012 1761 Industry Information | Data Science Institute 0.720294057599
3013 60424 Pradeep Naulia's Page - Data Science Central 0.720294057598
3014 28633 Data Scientist skillset, do you agree? - Data Science Central 0.720288172915
3015 18129 Defense Against the Dark Arts: 3Vs? 4Vs? - Data Science Central 0.720288172915
3016 16253 Data Diversity and Integration Trumps Big Data for Better Decision-making - Data Science Central 0.720279642122
3017 66038 Ramon Martinez's Page - Data Science Central 0.720241283863
3018 17618 Wilco van Ginkel's Page - Data Science Central 0.72023861857
3019 28583 Webinar: Non-Linear or Multi-Dimensional Approaches to Using News Sentiment - Data Science Central 0.72023861857
3020 1935 Objective-Based Data Monetization: A Enterprise Approach to Data Science (EDS) – Data Scientist Insi 0.720230510651
3021 57566 Nguyen Hoang's Page - Big Data News 0.720157934748
3022 60357 Martin Squires's Page - Data Scienc

3335 5265 That’s Data Science: Airbus Puts 10,000 Sensors in Every Single Wing! - Data Science Central 0.708984266817
3336 57634 Solving the Data Growth Crisis with Hadoop - Big Data News 0.708942101081
3337 66306 Data Analysis in R - Data Science Central 0.708921540346
3338 17961 Top 5 graph visualisation tools - Data Science Central 0.708873948453
3339 46752 Help Analyze Data in the Sun Seekers Challenge - Data Science Central 0.708839463963
3340 45708 Big Data and Data Science for Security and Fraud Detection 0.70883173495
3341 46809 Predictive Analytics Innovation: Connect with eBay, Dell & more - Data Science Central 0.708809971405
3342 57539 Bill Stevens's Page - Big Data News 0.708809971405
3343 5329 The Perfect Cheer - Data Science Central 0.7087953968
3344 57599 Big Data Explained in Less Than 2 Minutes - To Absolutely Anyone - Big Data News 0.70874667067
3345 57589 A Comprehensive List of Big Data Statistics - Big Data News 0.708687263224
3346 19255 segmentation - Data Plumbi

3480 5321 The Modern Data Architecture - DSC Webinar - Data Science Central 0.703984881696
3481 52255 M.S. in Data Science - Hybrid Option: Data Science M.S.: Data Science: Degrees & Certificates: Gradu 0.703984881696
3482 45289 Samsung 0.703855698305
3483 28686 Multivariate data visualization - Data Science Central 0.703765442792
3484 11125 Weekly Digest - January 26 - AnalyticBridge 0.703703703704
3485 46470 Real Time Data Solutions: Data Analyst 0.703663821036
3486 17809 Salary history and career path of a data scientist - Data Science Central 0.703663821036
3487 5237 Stop Hiring Data Scientists if you’re not ready for Data Science - Data Science Central 0.703658106284
3488 44804 DSTI 0.703623883874
3489 46482 Flipkart: Data Science interns, Data Scientists 0.703616657477
3490 36466 Jennifer Shin | The Big Data Hub 0.703575825375
3491 83025 KDnuggets Big Data Science Summer Reading List 0.703526470682
3492 46677 Book: Practical Machine Learning - A New Look At Anomaly Detection - Da

In [254]:
tmp = df[df['ds_sim'] > 0.70]
docs = tmp['body']
aG = DistanceGenerator(tmp)
sign = aG.calculate_signature(docs)

In [255]:
df['ds_sim'] =  df.key_words.map(lambda x: get_cosine(sign, x))

In [10]:
def get_summary(x):
    s = x.split()
    t = s[:50]
    ret = ' '.join(t) + '...'
    return ret

In [11]:
df['summary'] = df.body.map(lambda x: get_summary(x))

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86234 entries, 0 to 86233
Data columns (total 18 columns):
url                86234 non-null object
domain             86234 non-null object
body               86234 non-null object
title              86234 non-null object
key_words          86234 non-null object
sim                86234 non-null float64
dl_sim             86234 non-null float64
analytics_sim      86234 non-null float64
dm_sim             86234 non-null float64
ml_sim             86234 non-null float64
dsj_sim            86234 non-null float64
dsn_sim            86234 non-null float64
ba_sim             86234 non-null float64
ds_sim             86234 non-null float64
summary            86234 non-null object
signature          86234 non-null object
check_distances    86234 non-null float64
bd_sim             86234 non-null float64
dtypes: float64(11), object(7)
memory usage: 11.8+ MB


In [247]:
#dl_sim for deep learning
#analytics_sim for Analytics
#dm_sim for data mining
#ml_sim for machine learning
#dsj_sim for data science jobs
#dsn_sim for data science news
#ba_sim for big data


In [13]:
df.to_csv('../data/query/dsi_query_sim_v1.csv', encoding='utf-8', index=False)

In [241]:
df['t_sim'] =  df.key_words.map(lambda x: get_cosine(k1, x))
t_sim = df['t_sim'].values.argsort()[::-1]

In [243]:
idx = 0
for i in t_sim[1:20]:
    if df.iloc[i]['t_sim'] > 0.0:
        title = df.iloc[i]['title']
        sim_val = df.iloc[i]['t_sim']
        idx += 1
        print idx, i, title, sim_val

1 27056 Research in the School of Computing 0.33714498182
2 63160 WPI Professor Alexander Emanuel Receives Two Distinguished Honors from Institutions in his Native Ro 0.326983983188
3 59099 Research Areas 0.324066321999
4 64594 Why Ventana Research 0.320728904314
5 38359 MyQuirks Market Research dashboard 0.315680856769
6 27055 Research Groups and Centres 0.315330781073
7 64547 Ventana Research Introduces Business and Technology Research Agenda for 2015 0.310499237018
8 63161 Energy & Sustainability - WPI 0.30781804803
9 64570 Ventana Research Logo 0.307419447767
10 67498 University of Glasgow - Research & Knowledge Exchange 0.305324729759
11 23941 Ventana Research Logo 0.304251384429
12 35648 Sven Rill - http://www.iisys.de 0.302647815333
13 59098 Ethics 0.301639423419
14 77240 Faculty of Statistics - Research 0.301092241535
15 63164 Research Focus Areas - WPI 0.301081058513
16 64552 Benchmarking 0.301050652154
17 61943 Research - at University College Cork (UCC) 0.299929605689
18 586

In [244]:
len(df)

86412

In [246]:
print len(df) * len(df)

7467033744


In [ ]:
t0 = time()
for i in range(0, len())

In [273]:
from DistanceGenerator import DistanceGenerator 

ImportError: cannot import name DistanceGenerator

In [114]:
dG = DistanceGenerator(df)

In [105]:
print dG.calculate_distance(k1, kk2)

0.0225284262253


In [115]:
ret = dG.get_max_distances_idx(k1)

In [116]:
print ret[:200]

[ 3703 27056 63160 59099 64594 38359 27055 64547 63161 64570 67498 23941
 35648 59098 77240 63164 64552 61943 58656 64548 64578 64584 83006 23930
 42312 64528 29188 69882 64591 35285 17399 64521 64565 67490 23950 57224
 23944 47495 37905 64531 64577 63177 64568 20932 64559 23939 59102 64560
 23937  4511 23940 63568 15514 12613 29187 64576 17401 52840 61977 35403
  4127 73827   838 23936 23942  6261 63180 64571 64588 64587 71876 63168
  4129 39371 23938 64529 64562 64546 29153 64504 64585 64583 63848 15515
 64545 67516 68299 61254 33966 33740 64573 64540 64553  9193 21655 75807
  3578  9185 56801 54755 82018 63181 25364 64550 23949 20917 23947 80016
  1936 75808 74005 27100 64532 53565 64533 35284 73825 82730  9188 23943
 64542 17230 64541 37150 40691 12058 35416 81470  9136 23945 39212 82729
 64525 59100 25421 77512 63187 64522 64593  4867 35437 59926  8798 42313
 63189 23931 85008 64534 64589 21660 77715 23946 64558 42950 25369 69664
 64567 22920 25872 35435 40788 64564 84313 64575 64

In [7]:
df = pd.read_csv('../data/query/dsi_query_sim_v1.csv')

In [177]:
def get_summary(x):
    s = x.split()
    ret = ' '.join(s[:50]) + '...'
    return ret

In [11]:
for t in df['body'][:200]:
    print '---------'
    print get_summary(t)

---------
Organization Committee | ECMLPKDD2015 Skip to main content Toggle navigation Global Info Key Dates Organization Committee Sponsorship Discovery Challenges Past Conferences ECMLPKDD 2016 Program Preliminary Schedule Invited Speakers Workshop List Tutorial List Accepted Papers Submission Publication Model Journal Track Scientific Proceedings Track Industrial Proceedings Track Tutorials Workshops Nectar Track Demos...
---------
The Economist - World News, Politics, Economics, Business & Finance Our cookie policy has changed. Review our cookies policy for more details and to change your cookie preference. By continuing to browse this site you are agreeing to our use of cookies. More from The Economist The Economist digital editions...
---------
Films | The Economist Our cookie policy has changed. Review our cookies policy for more details and to change your cookie preference. By continuing to browse this site you are agreeing to our use of cookies. More from The Economist The Eco

In [178]:
df['summary'] = df.body.map(lambda x: get_summary(x))

In [180]:
df.head()

,url,domain,body,title,key_words,sim,dl_sim,analytics_sim,dm_sim,ml_sim,dsj_sim,dsn_sim,ba_sim,ds_sim,summary,signature,check_distances,bd_sim,topic
0,http://www.ecmlpkdd2015.org/global-info/organi...,ecmlpkdd2015.org,Organization Committee | ECMLPKDD2015 Skip to ...,Organization Committee | ECMLPKDD2015,"{u'industrial': 0.0108303249097, u'global': 0....",0.001585,0.000000,0.002048,0.001086,0.000000,0.000000,0.000000,0.0,0.000917,Organization Committee | ECMLPKDD2015 Skip to ...,"{u'industrial': 0.0108303249097, u'global': 0....",0.001770,0.001289,6
1,http://www.economist.com/,economist.com,"The Economist - World News, Politics, Economic...","The Economist - World News, Politics, Economic...","{u'radio': 0.0121951219512, u'economist': 0.13...",0.055683,0.033597,0.053106,0.035833,0.046231,0.062744,0.064590,0.0,0.066267,"The Economist - World News, Politics, Economic...","{u'radio': 0.0121951219512, u'economist': 0.13...",1.000000,0.036791,6
2,http://www.economist.com/films,economist.com,Films | The Economist Our cookie policy has ch...,Films | The Economist,"{u'tomorrows': 0.00847457627119, u'global': 0....",0.009322,0.001986,0.017344,0.003528,0.004963,0.006082,0.005733,0.0,0.006231,Films | The Economist Our cookie policy has ch...,"{u'tomorrows': 0.00847457627119, u'global': 0....",0.330666,0.002269,0
3,http://www.economist.com/news/christmas-specia...,economist.com,The king of con-men | The Economist Our cookie...,The king of con-men | The Economist,"{u'raised': 0.0268817204301, u'london': 0.0161...",0.020602,0.002845,0.023632,0.005913,0.004344,0.007028,0.006470,0.0,0.007238,The king of con-men | The Economist Our cookie...,"{u'raised': 0.0268817204301, u'london': 0.0161...",0.401691,0.012506,0
4,http://www.economist.com/news/christmas/215685...,economist.com,The song of Song | The Economist Our cookie po...,The song of Song | The Economist,"{u'chinas': 0.0259067357513, u'chinese': 0.041...",0.018020,0.000000,0.024232,0.005333,0.003918,0.004588,0.004124,0.0,0.004567,The song of Song | The Economist Our cookie po...,"{u'chinas': 0.0259067357513, u'chinese': 0.041...",0.377830,0.011307,6


In [260]:
df['signature'] = df['key_words']

In [17]:
df_slim = df[['url', 'title', 'summary',  'signature', 'dl_sim', 'analytics_sim', 'dm_sim', 'ml_sim', 'dsj_sim', 
              'dsn_sim', 'bd_sim', 'ds_sim', 'body']]

In [186]:
df.to_csv('../data/query/dsi_query_sim_v1.csv', encoding='utf-8', index=False)

In [18]:
df_slim.to_csv('../data/query/dsi_query_sim_slim_v1.csv', encoding='utf-8', index=False)

In [265]:
d = df_slim['url']
for t in d:
    if 'datareview.info' in d:
        print t

In [16]:
df_slim.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86234 entries, 0 to 86233
Data columns (total 12 columns):
url              86234 non-null object
title            86234 non-null object
summary          86234 non-null object
signature        86234 non-null object
dl_sim           86234 non-null float64
analytics_sim    86234 non-null float64
dm_sim           86234 non-null float64
ml_sim           86234 non-null float64
dsj_sim          86234 non-null float64
dsn_sim          86234 non-null float64
bd_sim           86234 non-null float64
ds_sim           86234 non-null float64
dtypes: float64(8), object(4)
memory usage: 7.9+ MB


In [67]:
class DistanceGenerator (object):
    def __init__(self, df):
        self.df = df
    
    def calculate_distance (self, v1, v2):
        vec1 = eval(v1)
        vec2 = eval(v2)
        #print type(vec1), type(vec2)
        #print vec2.keys()
        #print vec1.keys()
        intersection = set(vec1.keys()) & set(vec2.keys())
        #print intersection
        numerator = sum([vec1[x] * vec2[x] for x in intersection])

        sum1 = sum([vec1[x]**2 for x in vec1.keys()])
        sum2 = sum([vec2[x]**2 for x in vec2.keys()])
        denominator = math.sqrt(sum1) * math.sqrt(sum2)

        if not denominator:
            return 0.0
        else:
            return float(numerator) / denominator
    
    def get_max_distances_idx(self, vec):
        df['check_distances'] =  df.key_words.map(lambda x: self.calculate_distance(vec, x))
        dist = df['check_distances'].values.argsort()[::-1]
        return dist
    
    def dsi_title_letters_removal(self, x):
        s = re.sub('[^a-zA-Z]+', '', x)
        if(len(s) > 0):
            s = s.strip().lower()
            if s not in stop_words.ENGLISH_STOP_WORDS and s not in dsi_stop_words:
                return s
            else:
                return None
        return None

    def calculate_signature(self, docs):
        if len(docs) == 1:
            words = docs.split()
            flatten = [dsi_title_letters_removal(item) for item in words \
                       if dsi_title_letters_removal(item) != None]

        else:
            docs_split = [item.split() for item in docs]
            flatten = [self.dsi_title_letters_removal(item) for sublist in docs_split for item in sublist \
                       if self.dsi_title_letters_removal(item) != None]

        count = Counter(flatten)
        top_50 = count.most_common(50)
        words, cnt = zip(*top_50)
        tot = sum(cnt)
        fq = [float(i)*1.0/tot for i in cnt]
        sign = dict(zip(words, fq))
        return sign   



In [68]:
aG = DistanceGenerator(df)

In [69]:
aG.get_max_distances_idx(df.iloc[1]['signature'])

array([    1,  8544,  8551, ..., 31664, 31665, 43205])

In [78]:
text = 'data science jobs'
words = text.split()

In [71]:
len(words)

3

In [72]:
val = np.zeros(3)

In [79]:
val = [1.0/3 for item in words]

In [80]:
print val

[0.3333333333333333, 0.3333333333333333, 0.3333333333333333]


In [ ]:
for i in range(0, range(df)):
    sign = df.iloc[i]['signature']
    keys = sign.keys()
    if 'python'